# Introduction

---

This script aggregates features for the sales scrore model using three datasets: sales rank, review and price datasets. The features are generated per product category per brand. We take Avery brand, the product category_id1 'office-products/1069410' as an example.<br/>


The aggregation method first calculate weekly and monthly metrics for each product, e.g. average weekly sales rank. Then, based on the weekly and monthly metrics, we can further aggregate features for 'delta delta of log rank model', comparing the product in interest and the 10 products having the closest sales rank with it and calculating the difference in metrics between the two groups.<br/>


The logic of the aggregation method follows as below: <br/>
1) select product category <br/>
2) clean data and format date <br/> 
3) compute weekly/monthly sales rank, review & price metrics <br/>
4) aggregate weekly/monthly sales rank, review & price features for 'delta delta log rank model' <br/>


In [3]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import datetime, date, time, timedelta
from tqdm import tqdm
import re
from pandas.tseries.offsets import MonthEnd
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Read Datasets

In [0]:
month = ["March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
avery_sales_rank = pd.DataFrame()
for i in range(0,len(month)):
    sr_file_name = "SalesRankExport_Avery_" + month[i] + "2019.csv"
    df_bin = pd.read_csv(sr_file_name)
    df_bin = df_bin[['source', 'id', 'start_ts', 'end_ts', 'date', 'category_id1',
       'category_name1', 'category_rank1', 'category_id2', 'category_name2',
       'category_rank2', 'category_id3', 'category_name3', 'category_rank3',
       'category_id4', 'category_name4', 'category_rank4']]
    avery_sales_rank = pd.concat([avery_sales_rank, df_bin], ignore_index = True)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17,18,20,21,23,24,26,27,29,30,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
# add new sales rank data from Jan 2020 to March 2020
df_bin = pd.read_csv("SalesRankExport_f390988e-4679-471d-b9bb-b2a35062976b.csv")
df_bin = df_bin[['source', 'id', 'start_ts', 'end_ts', 'date', 'category_id1',
       'category_name1', 'category_rank1', 'category_id2', 'category_name2',
       'category_rank2', 'category_id3', 'category_name3', 'category_rank3',
       'category_id4', 'category_name4', 'category_rank4']]
avery_sales_rank = pd.concat([avery_sales_rank, df_bin], ignore_index = True)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,7,8,9,10,11,12,16,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
# review from March 2019 to Nov 2019
avery_review = pd.read_csv('BULK_all_56e3bb29-6796-44a0-8918-0cc355344eab.csv')
avery_review = avery_review[['product', 'source', 'date','author', 'verified', 'vine', 'stars',
                             'title', 'text', 'image']]

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3,8,11,12,24,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
# add new review data from Nov 2019 to March 2020
df_bin = pd.read_csv("Avery_BulkReviewsExport_Nov2019-Mar2020.csv")
df_bin = df_bin[['product', 'source','date','author', 'verified', 'vine', 'stars',
                             'title', 'text', 'image']]
avery_review = pd.concat([avery_review, df_bin], ignore_index = True)

In [0]:
# price data from March 2019 to Nov 2019
avery_price = pd.read_csv("BuyboxExport_eaf8d027-48c3-40c6-8147-7e41eded6c5c.csv")
avery_price = avery_price[avery_price['Source'] == 'amazon'][['Source', 'id', 'name',
                                                             'buyboxtime', 'price', 'high',
                                                       'low', 'stock', 'merchant' ]]

Archive:  /content/drive/My Drive/UC Davis Practicum/Pricebox Data/BuyboxExport_Avery_Mar2019-Nov2019_eaf8d027-48c3-40c6-8147-7e41eded6c5c.csv.zip
replace exports/BuyboxExport_eaf8d027-48c3-40c6-8147-7e41eded6c5c.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: exports/BuyboxExport_eaf8d027-48c3-40c6-8147-7e41eded6c5c.csv  


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,21,25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
# price data from Dec 2019 to Jan 2020
df_bin = pd.read_csv("BuyboxExport_8a64b831-a793-4cfc-acd0-9713b89ff12f.csv")
df_bin = df_bin[df_bin['Source'] == 'amazon'][['Source', 'id', 'name','buyboxtime', 'price', 'high',
                                                'low', 'stock', 'merchant' ]]
avery_price = pd.concat([avery_price, df_bin], ignore_index= True)

Archive:  /content/drive/My Drive/UC Davis Practicum/Pricebox Data/Avery_Prices_Dec1-Jan9-2020_BuyboxExport.csv.zip
replace exports/BuyboxExport_8a64b831-a793-4cfc-acd0-9713b89ff12f.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: exports/BuyboxExport_8a64b831-a793-4cfc-acd0-9713b89ff12f.csv  


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,21,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
# price data from Jan 2020 to Apr 2020
df_bin = pd.read_csv("BuyboxExport_5c5e6d9d-e9c4-47da-b68c-df0c270afcb8.csv")
df_bin = df_bin[df_bin['Source'] == 'amazon'][['Source', 'id', 'name','buyboxtime', 'price', 'high',
                                                'low', 'stock', 'merchant' ]]
avery_price = pd.concat([avery_price, df_bin], ignore_index= True)

Archive:  /content/drive/My Drive/UC Davis Practicum/Pricebox Data/Avery_Prices_Jan9-Apr72020_BuyboxExport_5c5e6d9d-e9c4-47da-b68c-df0c270afcb8.csv.zip
replace exports/BuyboxExport_5c5e6d9d-e9c4-47da-b68c-df0c270afcb8.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: exports/BuyboxExport_5c5e6d9d-e9c4-47da-b68c-df0c270afcb8.csv  


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,21,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Select Product Category

In [0]:
# count the number of products in each product cateogry of Avery brand
avery_prod_cat = avery_sales_rank.groupby(by='category_id1', as_index=False).agg({'id': pd.Series.nunique}).sort_values(by = 'id', ascending=False)
avery_prod_cat = avery_prod_cat.rename(columns = {"id": "num_products"})

In [0]:
avery_prod_cat.shape

(166, 2)

In [0]:
avery_prod_cat.head()

,category_id1,num_products
68,office-products/1069410,425
96,office-products/1069636,355
152,office-products/490930011,301
86,office-products/1069588,281
63,office-products/1069392,220


We select the category with the largest number of products: office-products/1069410, ink printer

In [0]:
category_id_str = 'office-products/490930011'
products = avery_sales_rank[avery_sales_rank['category_id1'] == \
                            category_id_str]['id'].drop_duplicates().reset_index(drop = True)

In [0]:
avery_sales_rank[avery_sales_rank['category_id1'] == \
                            'office-products/1069636']['category_name1'].unique()

array(['Office Products', 'Legal Indexes', 'Legal Index Dividers'],
      dtype=object)

In [0]:
products

0      B00006IBVA
1      B00006IBVG
2      B00006IBYA
3      B001HA2H58
4      B01FKQBAES
          ...    
296    B07XSMC679
297    B07XSMCV5V
298    B07XSMHBG1
299    B07XSNBL6Q
300    B07XSNFBGS
Name: id, Length: 301, dtype: object

In [0]:
# extract the rows and columns of the products of the selected category
avery_sales_rank_filtered = avery_sales_rank[avery_sales_rank['id'].isin(products)][['id','source','date', 
                                                                                    'category_id1', 'category_name1',
                                                                                    'category_rank1', 'category_id2', 
                                                                                     'category_name2', 'category_rank2']].reset_index(drop = True)

In [0]:
avery_review_filtered = avery_review[avery_review['product'].isin(products)][['product', 'source','date','author', 'verified', 'vine', 'stars',
                                                         'title', 'text', 'image']].reset_index(drop = True)

In [0]:
avery_price_filtered = avery_price[avery_price['id'].isin(products)][['id','buyboxtime','price', 'high',
                                                       'low', 'stock', 'merchant']].reset_index(drop = True)

## Deal with Duplicated Rows and Time Format

In [0]:
avery_sales_rank_filtered.drop_duplicates(inplace = True)

In [0]:
date_time = pd.to_datetime(avery_sales_rank_filtered['date'])
avery_sales_rank_filtered['month'] = date_time.dt.strftime('%Y-%m')
avery_sales_rank_filtered['month'] = pd.to_datetime(avery_sales_rank_filtered['month']) + MonthEnd(1)
avery_sales_rank_filtered['new_date'] = date_time.dt.strftime('%Y-%m-%d')
avery_sales_rank_filtered['new_date'] = pd.to_datetime(avery_sales_rank_filtered['new_date'])
avery_sales_rank_filtered['year'] = date_time.dt.strftime('%Y')
avery_sales_rank_filtered['week_of_year'] = date_time.dt.strftime('%V') # monday as the first day of week
avery_sales_rank_filtered['weekday'] = date_time.dt.strftime('%w') # return the weekday, Sunday is 0, Monday is 1

year_and_week column for time period recognition

In [0]:
avery_sales_rank_filtered['year_and_week'] = avery_sales_rank_filtered['year'] + '_'+ avery_sales_rank_filtered['week_of_year']

In [0]:
avery_review_filtered.drop_duplicates(inplace = True)

In [0]:
date_time = pd.to_datetime(avery_review_filtered['date'])
avery_review_filtered['month'] = date_time.dt.strftime('%Y-%m')
avery_review_filtered['month'] = pd.to_datetime(avery_review_filtered['month']) + MonthEnd(1)
avery_review_filtered['new_date'] = date_time.dt.strftime('%Y-%m-%d')
avery_review_filtered['new_date'] = pd.to_datetime(avery_review_filtered['new_date'])
avery_review_filtered['year'] = date_time.dt.strftime('%Y')
avery_review_filtered['week_of_year'] = date_time.dt.strftime('%V')
avery_review_filtered['weekday'] = date_time.dt.strftime('%w')
avery_review_filtered['year_and_week'] = avery_review_filtered['year'] + '_'+ avery_review_filtered['week_of_year']

In [0]:
avery_price_filtered.drop_duplicates(inplace = True)

In [0]:
date_time = pd.to_datetime(avery_price_filtered['buyboxtime'])
avery_price_filtered['month'] = date_time.dt.strftime('%Y-%m')
avery_price_filtered['month'] = pd.to_datetime(avery_price_filtered['month']) + MonthEnd(1)
avery_price_filtered['new_date'] = date_time.dt.strftime('%Y-%m-%d')
avery_price_filtered['new_date'] = pd.to_datetime(avery_price_filtered['new_date'])
avery_price_filtered['year'] = date_time.dt.strftime('%Y')
avery_price_filtered['week_of_year'] = date_time.dt.strftime('%V')
avery_price_filtered['weekday'] = date_time.dt.strftime('%w')
avery_price_filtered['year_and_week'] = avery_price_filtered['year'] + '_'+ avery_price_filtered['week_of_year']

In [0]:
avery_sales_rank_filtered.head()

,id,source,date,category_id1,category_name1,category_rank1,category_id2,category_name2,category_rank2,month,new_date,year,week_of_year,weekday,year_and_week
0,B00006IBVA,amazon,2019-03-11 20:16:58,office-products/490930011,Binder Index Dividers,137.0,office-products/1069564,Binder Accessories,445.0,2019-03-31,2019-03-11,2019,11,1,2019_11
1,B00006IBVA,amazon,2019-03-12 21:20:00,office-products/490930011,Office Products,69.0,office-products/1069564,Office Products,219.0,2019-03-31,2019-03-12,2019,11,2,2019_11
2,B00006IBVA,amazon,2019-03-13 19:38:52,office-products/490930011,Binder Index Dividers,113.0,office-products/1069564,Binder Accessories,345.0,2019-03-31,2019-03-13,2019,11,3,2019_11
3,B00006IBVA,amazon,2019-03-14 22:14:30,office-products/490930011,Binder Index Dividers,93.0,office-products/1069564,Binder Accessories,297.0,2019-03-31,2019-03-14,2019,11,4,2019_11
4,B00006IBVA,amazon,2019-03-15 21:40:11,office-products/490930011,Binder Index Dividers,121.0,office-products/1069564,Binder Accessories,398.0,2019-03-31,2019-03-15,2019,11,5,2019_11


In [0]:
avery_review_filtered.head()

,product,source,date,author,verified,vine,stars,title,text,image,month,new_date,year,week_of_year,weekday,year_and_week
0,B00006IBYA,amazon,2019-10-17 08:00,Christine N,True,NaN,4,A bit wide,Pockets on dividers are the best!!These are al...,No,2019-10-31,2019-10-17,2019,42,4,2019_42
1,B00006IBYA,amazon,2019-10-17 08:00,SlyShopper,True,NaN,5,Keeps my recipes separated,Got these to work with a binder for my recipe ...,No,2019-10-31,2019-10-17,2019,42,4,2019_42
2,B00006IBYA,amazon,2019-10-01 08:00,Vanessa Castillo,True,NaN,5,Cool,As expected,No,2019-10-31,2019-10-01,2019,40,2,2019_40
3,B00006IBYA,amazon,2019-09-24 08:00,Lauren,True,NaN,5,As advertised,They aren't super flimsy. Pleased with the value.,No,2019-09-30,2019-09-24,2019,39,2,2019_39
4,B00006IBYA,amazon,2019-09-22 08:00,marcella perez,True,NaN,5,Good purchase,Bought for my daughter for school. Was looking...,No,2019-09-30,2019-09-22,2019,38,0,2019_38


In [0]:
avery_price_filtered.head()

,id,buyboxtime,price,high,low,stock,merchant,month,new_date,year,week_of_year,weekday,year_and_week
0,B00006IBYA,2019-03-01 21:45:53,3.59,20.01,3.11,in,Amazon.com,2019-03-31,2019-03-01,2019,09,5,2019_09
1,B00006IBYA,2019-03-02 20:57:31,3.59,20.01,3.11,in,Amazon.com,2019-03-31,2019-03-02,2019,09,6,2019_09
2,B00006IBYA,2019-03-03 20:53:14,3.59,20.01,3.11,in,Amazon.com,2019-03-31,2019-03-03,2019,09,0,2019_09
3,B00006IBYA,2019-03-04 20:18:37,3.59,20.01,3.11,in,Amazon.com,2019-03-31,2019-03-04,2019,10,1,2019_10
4,B00006IBYA,2019-03-05 19:06:38,3.59,20.01,3.11,in,Amazon.com,2019-03-31,2019-03-05,2019,10,2,2019_10


## Create Dictionaries

Pick the last date in a month (in record) <br>
**For example**, 2020 Mar only has records until Mar 27, then use Mar 27 as the last date

In [0]:
# pick the rows on the last day of the month
avery_sales_rank_month = avery_sales_rank_filtered[avery_sales_rank_filtered.groupby(['id','month']).new_date.transform('max') 
                                                    == avery_sales_rank_filtered['new_date']].set_index('id', drop=True)

In [0]:
avery_sales_rank_month.head()

,source,date,category_id1,category_name1,category_rank1,category_id2,category_name2,category_rank2,month,new_date,year,week_of_year,weekday,year_and_week
id,,,,,,,,,,,,,,
B00006IBVA,amazon,2019-03-31 20:58:18,office-products/490930011,Binder Index Dividers,105.0,office-products,Office Products,28193.0,2019-03-31,2019-03-31,2019,13,0,2019_13
B00006IBVG,amazon,2019-03-31 20:18:32,office-products/490930011,Binder Index Dividers,20.0,office-products,Office Products,4462.0,2019-03-31,2019-03-31,2019,13,0,2019_13
B00006IBYA,amazon,2019-03-31 21:36:54,office-products/490930011,Binder Index Dividers,5.0,office-products,Office Products,986.0,2019-03-31,2019-03-31,2019,13,0,2019_13
B001HA2H58,amazon,2019-03-31 20:37:06,office-products/490930011,Binder Index Dividers,13.0,office-products,Office Products,3238.0,2019-03-31,2019-03-31,2019,13,0,2019_13
B004HLZ1FM,amazon,2019-03-31 21:26:05,office-products/1069628,Index Dividers,25.0,office-products,Office Products,4186.0,2019-03-31,2019-03-31,2019,13,0,2019_13


In [0]:
avery_sales_rank_month.columns

Index(['source', 'date', 'category_id1', 'category_name1', 'category_rank1',
       'category_id2', 'category_name2', 'category_rank2', 'month', 'new_date',
       'year', 'week_of_year', 'weekday', 'year_and_week'],
      dtype='object')

In [0]:
# create monthly sales rank dictionary
avery_sales_rank_dict_month = {}
for i in avery_sales_rank_month.index.unique():
    if type(avery_sales_rank_month.loc[i]) != pd.core.series.Series:
        avery_sales_rank_dict_month[i] = avery_sales_rank_month.loc[i, ['source', 'date', 'category_id1', 
                                                                    'category_name1', 'category_rank1',
                                                                    'category_id2', 'category_name2', 
                                                                    'category_rank2', 'month', 'new_date',
                                                                   'year', 'week_of_year', 'weekday', 'year_and_week']]
        avery_sales_rank_dict_month[i] = avery_sales_rank_dict_month[i].sort_values('date').reset_index(drop = True)

In [0]:
avery_sales_rank_dict_month[products[0]]

,source,date,category_id1,category_name1,category_rank1,category_id2,category_name2,category_rank2,month,new_date,year,week_of_year,weekday,year_and_week
0,amazon,2019-03-31 20:58:18,office-products/490930011,Binder Index Dividers,105.0,office-products,Office Products,28193.0,2019-03-31,2019-03-31,2019,13,0,2019_13
1,amazon,2019-04-30 20:05:29,office-products/490930011,Binder Index Dividers,145.0,office-products,Office Products,41208.0,2019-04-30,2019-04-30,2019,18,2,2019_18
2,amazon,2019-05-31 20:27:07,office-products/490930011,Binder Index Dividers,183.0,office-products,Office Products,45316.0,2019-05-31,2019-05-31,2019,22,5,2019_22
3,amazon,2019-06-30 19:17:48,office-products/490930011,Binder Index Dividers,139.0,office-products,Office Products,32552.0,2019-06-30,2019-06-30,2019,26,0,2019_26
4,amazon,2019-07-31 20:22:08,office-products/490930011,Binder Index Dividers,136.0,office-products,Office Products,23605.0,2019-07-31,2019-07-31,2019,31,3,2019_31
5,amazon,2019-08-31 20:22:19,office-products/490930011,Binder Index Dividers,28.0,office-products,Office Products,1367.0,2019-08-31,2019-08-31,2019,35,6,2019_35
6,amazon,2019-09-30 20:53:02,office-products/490930011,Binder Index Dividers,39.0,office-products,Office Products,5485.0,2019-09-30,2019-09-30,2019,40,1,2019_40
7,amazon,2019-10-31 22:10:33,office-products/490930011,Binder Index Dividers,36.0,office-products,Office Products,5320.0,2019-10-31,2019-10-31,2019,44,4,2019_44
8,amazon,2019-11-30 19:12:26,office-products/490930011,Binder Index Dividers,25.0,office-products,Office Products,9224.0,2019-11-30,2019-11-30,2019,48,6,2019_48
9,amazon,2019-12-31 19:09:48,office-products/490930011,Binder Index Dividers,24.0,office-products,Office Products,3913.0,2019-12-31,2019-12-31,2019,01,2,2019_01


In [0]:
# pick the rows on the Sunday of the week
avery_sales_rank_week = avery_sales_rank_filtered[\
                                                  avery_sales_rank_filtered['weekday'] == \
                                                  '0'].set_index('id', drop=True)

In [0]:
avery_sales_rank_week.columns

Index(['source', 'date', 'category_id1', 'category_name1', 'category_rank1',
       'category_id2', 'category_name2', 'category_rank2', 'month', 'new_date',
       'year', 'week_of_year', 'weekday', 'year_and_week'],
      dtype='object')

In [0]:
# create weekly sales rank dictionary
avery_sales_rank_dict_week = {}
for i in avery_sales_rank_week.index.unique():
    avery_sales_rank_dict_week[i] = avery_sales_rank_week.loc[i, ['source', 'date', 'category_id1', 
                                                                  'category_name1', 'category_rank1',
                                                                  'category_id2', 'category_name2', 
                                                                  'category_rank2', 'month', 'new_date',
                                                                  'year', 'week_of_year', 'weekday', 'year_and_week']]
    avery_sales_rank_dict_week[i] = avery_sales_rank_dict_week[i].sort_values('date').reset_index(drop = True)

In [0]:
avery_sales_rank_dict_week[products[0]]

,source,date,category_id1,category_name1,category_rank1,category_id2,category_name2,category_rank2,month,new_date,year,week_of_year,weekday,year_and_week
0,amazon,2019-03-17 19:20:08,office-products/490930011,Binder Index Dividers,108.0,office-products/1069564,Binder Accessories,361.0,2019-03-31,2019-03-17,2019,11,0,2019_11
1,amazon,2019-03-24 22:27:37,office-products/490930011,Binder Index Dividers,119.0,office-products/1069564,Binder Accessories,391.0,2019-03-31,2019-03-24,2019,12,0,2019_12
2,amazon,2019-03-31 20:58:18,office-products/490930011,Binder Index Dividers,105.0,office-products,Office Products,28193.0,2019-03-31,2019-03-31,2019,13,0,2019_13
3,amazon,2019-04-07 18:58:09,office-products/490930011,Binder Index Dividers,113.0,office-products,Office Products,33031.0,2019-04-30,2019-04-07,2019,14,0,2019_14
4,amazon,2019-04-14 22:53:41,office-products/490930011,Binder Index Dividers,86.0,office-products,Office Products,22984.0,2019-04-30,2019-04-14,2019,15,0,2019_15
5,amazon,2019-04-21 19:54:35,office-products/490930011,Binder Index Dividers,106.0,office-products,Office Products,28970.0,2019-04-30,2019-04-21,2019,16,0,2019_16
6,amazon,2019-04-28 18:51:30,office-products/490930011,Binder Index Dividers,111.0,office-products,Office Products,32895.0,2019-04-30,2019-04-28,2019,17,0,2019_17
7,amazon,2019-05-05 19:15:25,office-products/490930011,Binder Index Dividers,129.0,office-products,Office Products,35948.0,2019-05-31,2019-05-05,2019,18,0,2019_18
8,amazon,2019-05-12 19:21:39,office-products/490930011,Binder Index Dividers,128.0,office-products,Office Products,32666.0,2019-05-31,2019-05-12,2019,19,0,2019_19
9,amazon,2019-05-19 19:11:37,office-products/490930011,Binder Index Dividers,122.0,office-products,Office Products,30470.0,2019-05-31,2019-05-19,2019,20,0,2019_20


In [0]:
avery_sales_rank_daily = avery_sales_rank_filtered.set_index('id', drop=True)

In [0]:
avery_sales_rank_daily.columns

Index(['source', 'date', 'category_id1', 'category_name1', 'category_rank1',
       'category_id2', 'category_name2', 'category_rank2', 'month', 'new_date',
       'year', 'week_of_year', 'weekday', 'year_and_week'],
      dtype='object')

In [0]:
# create daily sales rank dictionary
avery_sales_rank_dict_daily = {}
for i in avery_sales_rank_daily.index.unique():
    avery_sales_rank_dict_daily[i] = avery_sales_rank_daily.loc[i, ['source', 'date', 'category_id1', 
                                                                  'category_name1', 'category_rank1',
                                                                  'category_id2', 'category_name2', 
                                                                  'category_rank2', 'month', 'new_date',
                                                                  'year', 'week_of_year', 'weekday', 'year_and_week']]
    avery_sales_rank_dict_daily[i] = avery_sales_rank_dict_daily[i].sort_values('date').reset_index(drop = True)

In [0]:
avery_sales_rank_dict_daily[products[0]].head()

,source,date,category_id1,category_name1,category_rank1,category_id2,category_name2,category_rank2,month,new_date,year,week_of_year,weekday,year_and_week
0,amazon,2019-03-11 20:16:58,office-products/490930011,Binder Index Dividers,137.0,office-products/1069564,Binder Accessories,445.0,2019-03-31,2019-03-11,2019,11,1,2019_11
1,amazon,2019-03-12 21:20:00,office-products/490930011,Office Products,69.0,office-products/1069564,Office Products,219.0,2019-03-31,2019-03-12,2019,11,2,2019_11
2,amazon,2019-03-13 19:38:52,office-products/490930011,Binder Index Dividers,113.0,office-products/1069564,Binder Accessories,345.0,2019-03-31,2019-03-13,2019,11,3,2019_11
3,amazon,2019-03-14 22:14:30,office-products/490930011,Binder Index Dividers,93.0,office-products/1069564,Binder Accessories,297.0,2019-03-31,2019-03-14,2019,11,4,2019_11
4,amazon,2019-03-15 21:40:11,office-products/490930011,Binder Index Dividers,121.0,office-products/1069564,Binder Accessories,398.0,2019-03-31,2019-03-15,2019,11,5,2019_11


In [0]:
avery_review_filtered = avery_review_filtered.set_index('product', drop = True)

In [0]:
avery_review_filtered.columns

Index(['source', 'date', 'author', 'verified', 'vine', 'stars', 'title',
       'text', 'image', 'month', 'new_date', 'year', 'week_of_year', 'weekday',
       'year_and_week'],
      dtype='object')

In [0]:
# create review dictionary
avery_review_dict = {}
for i in avery_review_filtered.index.unique():
    avery_review_dict[i] = avery_review_filtered.loc[i, ['source', 'date', 'author', 'verified', 'vine', 
                                                         'stars', 'title', 'text', 
                                                         'image', 'month', 'new_date',
                                                         'year', 'week_of_year', 'weekday', 'year_and_week']]
    if type(avery_review_dict[i]) != pd.core.series.Series:
        avery_review_dict[i] = avery_review_dict[i].sort_values('date').reset_index(drop = True) 

In [0]:
avery_review_dict[products[0]].head()

,source,date,author,verified,vine,stars,title,text,image,month,new_date,year,week_of_year,weekday,year_and_week
0,amazon,2012-12-26 08:00,busymom,True,NaN,5,"Avery WorkSaver Big Tab Insertable Dividers, 8...",The metal around the holes keep them from brea...,No,2012-12-31,2012-12-26,2012,52,3,2012_52
1,amazon,2013-03-08 08:00,Vine,True,NaN,5,Very satisfied,Good tabsI would recommend everyone to use itA...,No,2013-03-31,2013-03-08,2013,10,5,2013_10
2,amazon,2013-07-22 08:00,JessPlusOne,False,NaN,5,Affordable,These dividers are perfect for use in my class...,No,2013-07-31,2013-07-22,2013,30,1,2013_30
3,amazon,2013-08-15 08:00,Kindle Customer,False,NaN,5,BARGAIN!!!,BOUGHT THIS PRODUCT AS AN ADD-ON SO THE PRICE ...,No,2013-08-31,2013-08-15,2013,33,4,2013_33
4,amazon,2013-11-17 08:00,Leanne,True,NaN,5,Nice quality,These work great for indexing your notes in th...,No,2013-11-30,2013-11-17,2013,46,0,2013_46


In [0]:
avery_price_filtered = avery_price_filtered.set_index('id', drop = True)

In [0]:
avery_price_filtered.columns

Index(['buyboxtime', 'price', 'high', 'low', 'stock', 'merchant', 'month',
       'new_date', 'year', 'week_of_year', 'weekday', 'year_and_week'],
      dtype='object')

In [0]:
# create price dictionary
avery_price_dict = {}
for i in avery_price_filtered.index.unique():
    avery_price_dict[i] = avery_price_filtered.loc[i, ['buyboxtime', 'price', 'high', 'low', 'stock', 
                                                       'merchant', 'month', 'new_date', 'year',
                                                       'week_of_year', 'weekday', 'year_and_week']]
    avery_price_dict[i] = avery_price_dict[i].sort_values('buyboxtime').reset_index(drop = True)

In [0]:
avery_price_dict[products[0]].head()

,buyboxtime,price,high,low,stock,merchant,month,new_date,year,week_of_year,weekday,year_and_week
0,2019-03-01 21:59:49,NaN,13.34,0.99,3p,NaN,2019-03-31,2019-03-01,2019,09,5,2019_09
1,2019-03-02 21:39:37,NaN,13.34,0.99,3p,NaN,2019-03-31,2019-03-02,2019,09,6,2019_09
2,2019-03-03 20:23:12,NaN,13.34,0.99,3p,NaN,2019-03-31,2019-03-03,2019,09,0,2019_09
3,2019-03-04 20:39:18,NaN,13.34,0.99,3p,NaN,2019-03-31,2019-03-04,2019,10,1,2019_10
4,2019-03-05 19:18:59,NaN,13.34,0.99,3p,NaN,2019-03-31,2019-03-05,2019,10,2,2019_10


## Generate Weekly Metrics

### Aggregate Sales Rank Features

In [0]:
for i in tqdm(avery_sales_rank_dict_week):
    if i in avery_review_dict:
        for j in avery_sales_rank_dict_week[i].index.unique():
            if type(avery_review_dict[i]) != pd.core.series.Series:
                avery_sales_rank_dict_week[i].loc[j, 'avg_weekly_rank'] = \
                avery_sales_rank_dict_daily[i][(avery_sales_rank_dict_daily[i]['year_and_week'] ==
                                              avery_sales_rank_dict_week[i].loc[j, 'year_and_week'])]['category_rank1'].mean()
                avery_sales_rank_dict_week[i]['log_rank'] = \
                avery_sales_rank_dict_week[i]['category_rank1'].map(lambda x: math.log(x))
                avery_sales_rank_dict_week[i]['delta_rank'] = avery_sales_rank_dict_week[i]['log_rank'].diff()

100%|██████████| 301/301 [00:19<00:00, 15.05it/s]


In [0]:
avery_sales_rank_dict_week[products[0]].head()

,source,date,category_id1,category_name1,category_rank1,category_id2,category_name2,category_rank2,month,new_date,year,week_of_year,weekday,year_and_week,avg_weekly_rank,log_rank,delta_rank
0,amazon,2019-03-17 19:20:08,office-products/490930011,Binder Index Dividers,108.0,office-products/1069564,Binder Accessories,361.0,2019-03-31,2019-03-17,2019,11,0,2019_11,108.428571,4.682131,NaN
1,amazon,2019-03-24 22:27:37,office-products/490930011,Binder Index Dividers,119.0,office-products/1069564,Binder Accessories,391.0,2019-03-31,2019-03-24,2019,12,0,2019_12,124.428571,4.779123,0.096992
2,amazon,2019-03-31 20:58:18,office-products/490930011,Binder Index Dividers,105.0,office-products,Office Products,28193.0,2019-03-31,2019-03-31,2019,13,0,2019_13,130.428571,4.653960,-0.125163
3,amazon,2019-04-07 18:58:09,office-products/490930011,Binder Index Dividers,113.0,office-products,Office Products,33031.0,2019-04-30,2019-04-07,2019,14,0,2019_14,139.714286,4.727388,0.073427
4,amazon,2019-04-14 22:53:41,office-products/490930011,Binder Index Dividers,86.0,office-products,Office Products,22984.0,2019-04-30,2019-04-14,2019,15,0,2019_15,134.000000,4.454347,-0.273041


### Aggregate Star Rating Features

In [0]:
# stars_one_week: the average star rating of the most recent one week
for i in tqdm(avery_sales_rank_dict_week):
    if i in avery_review_dict:
        for j in avery_sales_rank_dict_week[i].index.unique():
            if type(avery_review_dict[i]) != pd.core.series.Series:
                avery_sales_rank_dict_week[i].loc[j, 'stars_recent_10'] = \
                avery_review_dict[i][avery_review_dict[i]['new_date'] <= \
                                     avery_sales_rank_dict_week[i].loc[j, 'new_date']].tail(10)['stars'].mean()
                
                avery_sales_rank_dict_week[i].loc[j, "stars_one_week"] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_week[i].loc[j, 'new_date']) &\
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_week[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 7)))]['stars'].mean()
                # stars_pri_1stweek: the average star rating in the first week prior to this week
                # stars_pri_1stweek is also the stars_one_week lagged by one
                avery_sales_rank_dict_week[i]['stars_pri_1stweek'] = \
                avery_sales_rank_dict_week[i]['stars_one_week'].shift(periods=1)
                avery_sales_rank_dict_week[i]['stars_pri_2ndweek'] = \
                avery_sales_rank_dict_week[i]['stars_one_week'].shift(periods=2)
                avery_sales_rank_dict_week[i]['stars_pri_3rdweek'] = \
                avery_sales_rank_dict_week[i]['stars_one_week'].shift(periods=3)
                avery_sales_rank_dict_week[i]['stars_pri_4thweek'] = \
                avery_sales_rank_dict_week[i]['stars_one_week'].shift(periods=4)
                

100%|██████████| 301/301 [00:45<00:00,  6.56it/s]


In [0]:
# stars_two_weeks: the average star rating of the most recent two weeks
for i in tqdm(avery_sales_rank_dict_week):
    if i in avery_review_dict:
        for j in avery_sales_rank_dict_week[i].index.unique():
            if type(avery_review_dict[i]) != pd.core.series.Series:
                avery_sales_rank_dict_week[i].loc[j, "stars_two_weeks"] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_week[i].loc[j, 'new_date']) &\
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_week[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 14)))]['stars'].mean()
                avery_sales_rank_dict_week[i]['stars_two_weeks_lag1'] = \
                avery_sales_rank_dict_week[i]['stars_two_weeks'].shift(periods=1)
                avery_sales_rank_dict_week[i]['stars_two_weeks_lag2'] = \
                avery_sales_rank_dict_week[i]['stars_two_weeks'].shift(periods=2)

100%|██████████| 301/301 [00:24<00:00, 12.19it/s]


In [0]:
for i in tqdm(avery_sales_rank_dict_week):
    if i in avery_review_dict:
        for j in avery_sales_rank_dict_week[i].index.unique():
            if type(avery_review_dict[i]) != pd.core.series.Series:
                avery_sales_rank_dict_week[i].loc[j, "stars_one_month"] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_week[i].loc[j, 'new_date']) &\
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_week[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 30)))]['stars'].mean()
                avery_sales_rank_dict_week[i].loc[j, "stars_two_months"] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_week[i].loc[j, 'new_date']) &\
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_week[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 60)))]['stars'].mean()
                avery_sales_rank_dict_week[i].loc[j, "stars_one_quarter"] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_week[i].loc[j, 'new_date']) &\
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_week[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 90)))]['stars'].mean()
                avery_sales_rank_dict_week[i].loc[j, "stars_avg"] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_week[i].loc[j, 'new_date'])]['stars'].mean()
                

100%|██████████| 301/301 [01:05<00:00,  4.58it/s]


In [0]:
avery_sales_rank_dict_week[products[0]].head()

,source,date,category_id1,category_name1,category_rank1,category_id2,category_name2,category_rank2,month,new_date,year,week_of_year,weekday,year_and_week,avg_weekly_rank,log_rank,delta_rank,stars_recent_10,stars_one_week,stars_pri_1stweek,stars_pri_2ndweek,stars_pri_3rdweek,stars_pri_4thweek,stars_two_weeks,stars_two_weeks_lag1,stars_two_weeks_lag2,stars_one_month,stars_two_months,stars_one_quarter,stars_avg
0,amazon,2019-03-17 19:20:08,office-products/490930011,Binder Index Dividers,108.0,office-products/1069564,Binder Accessories,361.0,2019-03-31,2019-03-17,2019,11,0,2019_11,108.428571,4.682131,NaN,5.0,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,5.0,5.0,4.520325
1,amazon,2019-03-24 22:27:37,office-products/490930011,Binder Index Dividers,119.0,office-products/1069564,Binder Accessories,391.0,2019-03-31,2019-03-24,2019,12,0,2019_12,124.428571,4.779123,0.096992,5.0,NaN,5.0,NaN,NaN,NaN,5.0,5.0,NaN,5.0,5.0,5.0,4.520325
2,amazon,2019-03-31 20:58:18,office-products/490930011,Binder Index Dividers,105.0,office-products,Office Products,28193.0,2019-03-31,2019-03-31,2019,13,0,2019_13,130.428571,4.653960,-0.125163,5.0,NaN,NaN,5.0,NaN,NaN,NaN,5.0,5.0,5.0,5.0,5.0,4.520325
3,amazon,2019-04-07 18:58:09,office-products/490930011,Binder Index Dividers,113.0,office-products,Office Products,33031.0,2019-04-30,2019-04-07,2019,14,0,2019_14,139.714286,4.727388,0.073427,5.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,5.0,5.0,5.0,5.0,4.520325
4,amazon,2019-04-14 22:53:41,office-products/490930011,Binder Index Dividers,86.0,office-products,Office Products,22984.0,2019-04-30,2019-04-14,2019,15,0,2019_15,134.000000,4.454347,-0.273041,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,5.0,5.0,4.520325


### Aggregate Quantity of Reviews Features

In [0]:
for i in tqdm(avery_sales_rank_dict_week):
    if i in avery_review_dict:
        for j in avery_sales_rank_dict_week[i].index.unique():
            if type(avery_review_dict[i]) != pd.core.series.Series:
                avery_sales_rank_dict_week[i].loc[j, "num_review"] = \
                avery_review_dict[i][avery_review_dict[i]['new_date'] <= \
                                     avery_sales_rank_dict_week[i].loc[j, 'new_date']]['new_date'].count()
               
                avery_sales_rank_dict_week[i].loc[j, 'num_review_one_week'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_week[i].loc[j,'new_date'] - \
                                       timedelta(days = 7)))]['new_date'].count()
                
                avery_sales_rank_dict_week[i]['num_review_one_week_lag1'] = \
                avery_sales_rank_dict_week[i]['num_review_one_week'].shift(periods=1)
                avery_sales_rank_dict_week[i]['num_review_one_week_lag2'] = \
                avery_sales_rank_dict_week[i]['num_review_one_week'].shift(periods=2)
                avery_sales_rank_dict_week[i]['num_review_one_week_lag3'] = \
                avery_sales_rank_dict_week[i]['num_review_one_week'].shift(periods=3)
                avery_sales_rank_dict_week[i]['num_review_one_week_lag4'] = \
                avery_sales_rank_dict_week[i]['num_review_one_week'].shift(periods=4)
                
                avery_sales_rank_dict_week[i].loc[j, 'num_review_two_weeks'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_week[i].loc[j,'new_date'] - \
                                       timedelta(days = 14)))]['new_date'].count()
                
                avery_sales_rank_dict_week[i]['num_review_two_weeks_lag1'] = \
                avery_sales_rank_dict_week[i]['num_review_two_weeks'].shift(periods=1)
                avery_sales_rank_dict_week[i]['num_review_two_weeks_lag2'] = \
                avery_sales_rank_dict_week[i]['num_review_two_weeks'].shift(periods=2)
                
                avery_sales_rank_dict_week[i].loc[j, 'num_review_one_month'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_week[i].loc[j,'new_date'] - \
                                       timedelta(days = 30)))]['new_date'].count()
                
                avery_sales_rank_dict_week[i].loc[j, 'num_review_two_months'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_week[i].loc[j,'new_date'] - \
                                       timedelta(days = 60)))]['new_date'].count()
                
                avery_sales_rank_dict_week[i].loc[j, 'num_review_one_quarter'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_week[i].loc[j,'new_date'] - \
                                       timedelta(days = 90)))]['new_date'].count()

100%|██████████| 301/301 [01:49<00:00,  2.75it/s]


In [0]:
avery_sales_rank_dict_week[products[0]].head()

,source,date,category_id1,category_name1,category_rank1,category_id2,category_name2,category_rank2,month,new_date,year,week_of_year,weekday,year_and_week,avg_weekly_rank,log_rank,delta_rank,stars_recent_10,stars_one_week,stars_pri_1stweek,stars_pri_2ndweek,stars_pri_3rdweek,stars_pri_4thweek,stars_two_weeks,stars_two_weeks_lag1,stars_two_weeks_lag2,stars_one_month,stars_two_months,stars_one_quarter,stars_avg,num_review,num_review_one_week,num_review_one_week_lag1,num_review_one_week_lag2,num_review_one_week_lag3,num_review_one_week_lag4,num_review_two_weeks,num_review_two_weeks_lag1,num_review_two_weeks_lag2,num_review_one_month,num_review_two_months,num_review_one_quarter
0,amazon,2019-03-17 19:20:08,office-products/490930011,Binder Index Dividers,108.0,office-products/1069564,Binder Accessories,361.0,2019-03-31,2019-03-17,2019,11,0,2019_11,108.428571,4.682131,NaN,5.0,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,5.0,5.0,4.520325,123.0,1.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,3.0,3.0,3.0
1,amazon,2019-03-24 22:27:37,office-products/490930011,Binder Index Dividers,119.0,office-products/1069564,Binder Accessories,391.0,2019-03-31,2019-03-24,2019,12,0,2019_12,124.428571,4.779123,0.096992,5.0,NaN,5.0,NaN,NaN,NaN,5.0,5.0,NaN,5.0,5.0,5.0,4.520325,123.0,0.0,1.0,NaN,NaN,NaN,1.0,2.0,NaN,2.0,3.0,3.0
2,amazon,2019-03-31 20:58:18,office-products/490930011,Binder Index Dividers,105.0,office-products,Office Products,28193.0,2019-03-31,2019-03-31,2019,13,0,2019_13,130.428571,4.653960,-0.125163,5.0,NaN,NaN,5.0,NaN,NaN,NaN,5.0,5.0,5.0,5.0,5.0,4.520325,123.0,0.0,0.0,1.0,NaN,NaN,0.0,1.0,2.0,2.0,3.0,3.0
3,amazon,2019-04-07 18:58:09,office-products/490930011,Binder Index Dividers,113.0,office-products,Office Products,33031.0,2019-04-30,2019-04-07,2019,14,0,2019_14,139.714286,4.727388,0.073427,5.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,5.0,5.0,5.0,5.0,4.520325,123.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,1.0,2.0,3.0,3.0
4,amazon,2019-04-14 22:53:41,office-products/490930011,Binder Index Dividers,86.0,office-products,Office Products,22984.0,2019-04-30,2019-04-14,2019,15,0,2019_15,134.000000,4.454347,-0.273041,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,5.0,5.0,4.520325,123.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,3.0


### Aggregate Quantity of Reviews with Extreme Ratings

In [0]:
for i in tqdm(avery_sales_rank_dict_week):
    if i in avery_review_dict:
        for j in avery_sales_rank_dict_week[i].index.unique():
            if type(avery_review_dict[i]) != pd.core.series.Series:
                avery_sales_rank_dict_week[i].loc[j, 'num_1star_one_week'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_week[i].loc[j,'new_date'] - timedelta(days = 7))) & \
                                     (avery_review_dict[i]['stars'] == 1)]['new_date'].count()
                
                avery_sales_rank_dict_week[i]['num_1star_one_week_lag1'] = \
                avery_sales_rank_dict_week[i]['num_1star_one_week'].shift(periods=1)
                avery_sales_rank_dict_week[i]['num_1star_one_week_lag2'] = \
                avery_sales_rank_dict_week[i]['num_1star_one_week'].shift(periods=2)
                avery_sales_rank_dict_week[i]['num_1star_one_week_lag3'] = \
                avery_sales_rank_dict_week[i]['num_1star_one_week'].shift(periods=3)
                avery_sales_rank_dict_week[i]['num_1star_one_week_lag4'] = \
                avery_sales_rank_dict_week[i]['num_1star_one_week'].shift(periods=4)
                
                avery_sales_rank_dict_week[i].loc[j, 'num_1star_two_weeks'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_week[i].loc[j,'new_date'] - timedelta(days = 14))) & \
                                     (avery_review_dict[i]['stars'] == 1)]['new_date'].count()
                
                avery_sales_rank_dict_week[i]['num_1star_two_weeks_lag1'] = \
                avery_sales_rank_dict_week[i]['num_1star_two_weeks'].shift(periods=1)
                avery_sales_rank_dict_week[i]['num_1star_two_weeks_lag2'] = \
                avery_sales_rank_dict_week[i]['num_1star_two_weeks'].shift(periods=2)
                
                avery_sales_rank_dict_week[i].loc[j, 'num_1star_one_month'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_week[i].loc[j,'new_date'] - timedelta(days = 30))) & \
                                     (avery_review_dict[i]['stars'] == 1)]['new_date'].count()
          
                avery_sales_rank_dict_week[i].loc[j, 'num_1star_two_months'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_week[i].loc[j,'new_date'] - timedelta(days = 60))) & \
                                     (avery_review_dict[i]['stars'] == 1)]['new_date'].count()
                
                avery_sales_rank_dict_week[i].loc[j, 'num_1star_one_quarter'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_week[i].loc[j,'new_date'] - timedelta(days = 90))) & \
                                     (avery_review_dict[i]['stars'] == 1)]['new_date'].count()
                
                avery_sales_rank_dict_week[i].loc[j, 'num_1star'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['stars'] == 1)]['new_date'].count()
                
                # quantity of 5-star reviews
                avery_sales_rank_dict_week[i].loc[j, 'num_5star_one_week'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_week[i].loc[j,'new_date'] - timedelta(days = 7))) & \
                                     (avery_review_dict[i]['stars'] == 5)]['new_date'].count()
                
                avery_sales_rank_dict_week[i]['num_5star_one_week_lag1'] = \
                avery_sales_rank_dict_week[i]['num_5star_one_week'].shift(periods=1)
                avery_sales_rank_dict_week[i]['num_5star_one_week_lag2'] = \
                avery_sales_rank_dict_week[i]['num_5star_one_week'].shift(periods=2)
                avery_sales_rank_dict_week[i]['num_5star_one_week_lag3'] = \
                avery_sales_rank_dict_week[i]['num_5star_one_week'].shift(periods=3)
                avery_sales_rank_dict_week[i]['num_5star_one_week_lag4'] = \
                avery_sales_rank_dict_week[i]['num_5star_one_week'].shift(periods=4)
                
                avery_sales_rank_dict_week[i].loc[j, 'num_5star_two_weeks'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_week[i].loc[j,'new_date'] - timedelta(days = 14))) & \
                                     (avery_review_dict[i]['stars'] == 5)]['new_date'].count()
                
                avery_sales_rank_dict_week[i]['num_5star_two_weeks_lag1'] = \
                avery_sales_rank_dict_week[i]['num_5star_two_weeks'].shift(periods=1)
                avery_sales_rank_dict_week[i]['num_5star_two_weeks_lag2'] = \
                avery_sales_rank_dict_week[i]['num_5star_two_weeks'].shift(periods=2)
                
                avery_sales_rank_dict_week[i].loc[j, 'num_5star_one_month'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_week[i].loc[j,'new_date'] - timedelta(days = 30))) & \
                                     (avery_review_dict[i]['stars'] == 5)]['new_date'].count()
          
                avery_sales_rank_dict_week[i].loc[j, 'num_5star_two_months'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_week[i].loc[j,'new_date'] - timedelta(days = 60))) & \
                                     (avery_review_dict[i]['stars'] == 5)]['new_date'].count()
                
                avery_sales_rank_dict_week[i].loc[j, 'num_5star_one_quarter'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_week[i].loc[j,'new_date'] - timedelta(days = 90))) & \
                                     (avery_review_dict[i]['stars'] == 5)]['new_date'].count()
                
                avery_sales_rank_dict_week[i].loc[j, 'num_5star'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['stars'] == 5)]['new_date'].count()
                

100%|██████████| 301/301 [03:58<00:00,  1.26it/s]


In [0]:
avery_sales_rank_dict_week[products[0]].head()

,source,date,category_id1,category_name1,category_rank1,category_id2,category_name2,category_rank2,month,new_date,year,week_of_year,weekday,year_and_week,avg_weekly_rank,log_rank,delta_rank,stars_recent_10,stars_one_week,stars_pri_1stweek,stars_pri_2ndweek,stars_pri_3rdweek,stars_pri_4thweek,stars_two_weeks,stars_two_weeks_lag1,stars_two_weeks_lag2,stars_one_month,stars_two_months,stars_one_quarter,stars_avg,num_review,num_review_one_week,num_review_one_week_lag1,num_review_one_week_lag2,num_review_one_week_lag3,num_review_one_week_lag4,num_review_two_weeks,num_review_two_weeks_lag1,num_review_two_weeks_lag2,num_review_one_month,num_review_two_months,num_review_one_quarter,num_1star_one_week,num_1star_one_week_lag1,num_1star_one_week_lag2,num_1star_one_week_lag3,num_1star_one_week_lag4,num_1star_two_weeks,num_1star_two_weeks_lag1,num_1star_two_weeks_lag2,num_1star_one_month,num_1star_two_months,num_1star_one_quarter,num_1star,num_5star_one_week,num_5star_one_week_lag1,num_5star_one_week_lag2,num_5star_one_week_lag3,num_5star_one_week_lag4,num_5star_two_weeks,num_5star_two_weeks_lag1,num_5star_two_weeks_lag2,num_5star_one_month,num_5star_two_months,num_5star_one_quarter,num_5star
0,amazon,2019-03-17 19:20:08,office-products/490930011,Binder Index Dividers,108.0,office-products/1069564,Binder Accessories,361.0,2019-03-31,2019-03-17,2019,11,0,2019_11,108.428571,4.682131,NaN,5.0,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,5.0,5.0,4.520325,123.0,1.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,3.0,3.0,3.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,7.0,1.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,3.0,3.0,3.0,93.0
1,amazon,2019-03-24 22:27:37,office-products/490930011,Binder Index Dividers,119.0,office-products/1069564,Binder Accessories,391.0,2019-03-31,2019-03-24,2019,12,0,2019_12,124.428571,4.779123,0.096992,5.0,NaN,5.0,NaN,NaN,NaN,5.0,5.0,NaN,5.0,5.0,5.0,4.520325,123.0,0.0,1.0,NaN,NaN,NaN,1.0,2.0,NaN,2.0,3.0,3.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,7.0,0.0,1.0,NaN,NaN,NaN,1.0,2.0,NaN,2.0,3.0,3.0,93.0
2,amazon,2019-03-31 20:58:18,office-products/490930011,Binder Index Dividers,105.0,office-products,Office Products,28193.0,2019-03-31,2019-03-31,2019,13,0,2019_13,130.428571,4.653960,-0.125163,5.0,NaN,NaN,5.0,NaN,NaN,NaN,5.0,5.0,5.0,5.0,5.0,4.520325,123.0,0.0,0.0,1.0,NaN,NaN,0.0,1.0,2.0,2.0,3.0,3.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,1.0,NaN,NaN,0.0,1.0,2.0,2.0,3.0,3.0,93.0
3,amazon,2019-04-07 18:58:09,office-products/490930011,Binder Index Dividers,113.0,office-products,Office Products,33031.0,2019-04-30,2019-04-07,2019,14,0,2019_14,139.714286,4.727388,0.073427,5.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,5.0,5.0,5.0,5.0,4.520325,123.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,1.0,2.0,3.0,3.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,1.0,2.0,3.0,3.0,93.0
4,amazon,2019-04-14 22:53:41,office-products/490930011,Binder Index Dividers,86.0,office-products,Office Products,22984.0,2019-04-30,2019-04-14,2019,15,0,2019_15,134.000000,4.454347,-0.273041,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,5.0,5.0,4.520325,123.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,3.0,93.0


### Aggregate Quality of Reviews Features

In [0]:
for i in tqdm(avery_sales_rank_dict_week):
    if i in avery_review_dict:
        for j in avery_sales_rank_dict_week[i].index.unique():
            if type(avery_review_dict[i]) != pd.core.series.Series:
                # aggregate word count related features
                wc_bin = avery_review_dict[i][avery_review_dict[i]['new_date'] <= \
                        avery_sales_rank_dict_week[i].loc[j, 'new_date']]['text'].tolist()
                wc_bin_str = ''.join(map(str, wc_bin))
                if len(wc_bin) > 0:
                    avery_sales_rank_dict_week[i].loc[j, 'avg_word_count'] = \
                    len(wc_bin_str.split())/len(wc_bin)
                else: 
                    avery_sales_rank_dict_week[i].loc[j, 'avg_word_count'] = 0
                    
                #  'avg_word_count_10' is the average word count of the recent 10 reviews
                wc2_bin = avery_review_dict[i][avery_review_dict[i]['new_date'] <= \
                    avery_sales_rank_dict_week[i].loc[j, 'new_date']].tail(10)['text'].tolist()
                wc2_bin_str = ''.join(map(str, wc2_bin))
                if len(wc2_bin) > 0:
                    avery_sales_rank_dict_week[i].loc[j, 'avg_word_count_10'] = \
                    len(wc2_bin_str.split())/len(wc2_bin)
                else:
                    avery_sales_rank_dict_week[i].loc[j, 'avg_word_count_10'] = 0
                
                # perc_named_author: the percentage of reviews written by non-anonymous authors to the date
                avery_sales_rank_dict_week[i].loc[j, 'perc_named_author'] = \
                avery_review_dict[i][
                    (avery_review_dict[i]['new_date'] <= avery_sales_rank_dict_week[i].loc[j,'new_date']) & \
                    (avery_review_dict[i]['author'].str.contains('[Cc]ustomer') == False)
                ]['new_date'].count() \
                    / avery_review_dict[i][avery_review_dict[i]['new_date'] \
                                <= avery_sales_rank_dict_week[i].loc[j,'new_date']]['new_date'].count()
                
                # 'perc_verified': the percentage of verified reviews to the date
                avery_sales_rank_dict_week[i].loc[j, 'perc_verified'] = \
                    (avery_review_dict[i][
                    (avery_review_dict[i]['new_date'] <= avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['verified'] == True)]['new_date'].count()) / \
                    (avery_review_dict[i][avery_review_dict[i]['new_date'] < \
                                        avery_sales_rank_dict_week[i].loc[j, 'new_date']]['new_date'].count()) 
                
                avery_sales_rank_dict_week[i].loc[j, 'perc_verified_one_quarter'] = \
                    (avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_week[i].loc[j,'new_date'] - timedelta(days = 90))) & \
                                          (avery_review_dict[i]['verified'] == True)]['new_date'].count()) / \
                    (avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_week[i].loc[j,'new_date'] - \
                                                          timedelta(days = 90)))]['new_date'].count()) 
            
                avery_sales_rank_dict_week[i].loc[j, 'perc_verified_one_month'] = \
                    (avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_week[i].loc[j,'new_date'] - timedelta(days = 30))) & \
                                          (avery_review_dict[i]['verified'] == True)]['new_date'].count()) / \
                    (avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_week[i].loc[j,'new_date'] - \
                                                          timedelta(days = 30)))]['new_date'].count()) 
                
                # 'num_image': number of reviews with image
                avery_sales_rank_dict_week[i].loc[j, 'num_image'] = \
                    (avery_review_dict[i][
                    (avery_review_dict[i]['new_date'] <= avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['image'] == 'Yes')]['new_date'].count())
                
                avery_sales_rank_dict_week[i].loc[j, 'num_image_one_quarter'] = \
                    (avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_week[i].loc[j,'new_date'] - timedelta(days = 90))) & \
                    (avery_review_dict[i]['image'] == 'Yes')]['new_date'].count())
                
                avery_sales_rank_dict_week[i].loc[j, 'num_image_one_month'] = \
                    (avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_week[i].loc[j,'new_date'] - timedelta(days = 30))) & \
                    (avery_review_dict[i]['image'] == 'Yes')]['new_date'].count())
                
                # 'num_vine': number of reviews by Vine customers
                avery_sales_rank_dict_week[i].loc[j, 'num_vine'] = \
                    (avery_review_dict[i][
                    (avery_review_dict[i]['new_date'] <= avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['vine'] == True)]['new_date'].count())
                
                avery_sales_rank_dict_week[i].loc[j, 'num_vine_one_quarter'] = \
                    (avery_review_dict[i][(
                    avery_review_dict[i]['new_date'] < avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_week[i].loc[j,'new_date'] - timedelta(days = 90))) & \
                    (avery_review_dict[i]['vine'] == True)]['new_date'].count())
                
                avery_sales_rank_dict_week[i].loc[j, 'num_vine_one_month'] = \
                    (avery_review_dict[i][(
                    avery_review_dict[i]['new_date'] < avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_week[i].loc[j,'new_date'] - timedelta(days = 30))) & \
                    (avery_review_dict[i]['vine'] == True)]['new_date'].count())
                

  0%|          | 0/301 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in long_scalars
 25%|██▌       | 76/301 [01:26<04:16,  1.14s/it]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in long_scalars
100%|██████████| 301/301 [03:56<00:00,  1.27it/s]


### Aggregate Price Features

In [0]:
for i in tqdm(avery_sales_rank_dict_week):
    if i in avery_review_dict:
        for j in avery_sales_rank_dict_week[i].index.unique():
            if type(avery_review_dict[i]) != pd.core.series.Series:
                avery_sales_rank_dict_week[i].loc[j, 'avg_weekly_price'] = avery_price_dict[i][ \
                    (avery_price_dict[i]['year_and_week'] == avery_sales_rank_dict_week[i].loc[j, 'year_and_week'])]['price'].mean()
                
                avery_sales_rank_dict_week[i].loc[j, 'avg_monthly_price'] = avery_price_dict[i][(\
                    avery_price_dict[i]['new_date'] < avery_sales_rank_dict_week[i].loc[j, 'new_date']) & \
                    (avery_price_dict[i]['new_date'] >= (avery_sales_rank_dict_week[i].loc[j,'new_date'] - timedelta(days = 30)))\
                                                                                                ]['price'].mean()

100%|██████████| 301/301 [00:29<00:00, 10.05it/s]


In [0]:
avery_sales_rank_dict_week[products[0]].head()

,source,date,category_id1,category_name1,category_rank1,category_id2,category_name2,category_rank2,month,new_date,year,week_of_year,weekday,year_and_week,avg_weekly_rank,log_rank,delta_rank,stars_recent_10,stars_one_week,stars_pri_1stweek,stars_pri_2ndweek,stars_pri_3rdweek,stars_pri_4thweek,stars_two_weeks,stars_two_weeks_lag1,stars_two_weeks_lag2,stars_one_month,stars_two_months,stars_one_quarter,stars_avg,num_review,num_review_one_week,num_review_one_week_lag1,num_review_one_week_lag2,num_review_one_week_lag3,num_review_one_week_lag4,num_review_two_weeks,num_review_two_weeks_lag1,num_review_two_weeks_lag2,num_review_one_month,num_review_two_months,num_review_one_quarter,num_1star_one_week,num_1star_one_week_lag1,num_1star_one_week_lag2,num_1star_one_week_lag3,num_1star_one_week_lag4,num_1star_two_weeks,num_1star_two_weeks_lag1,num_1star_two_weeks_lag2,num_1star_one_month,num_1star_two_months,num_1star_one_quarter,num_1star,num_5star_one_week,num_5star_one_week_lag1,num_5star_one_week_lag2,num_5star_one_week_lag3,num_5star_one_week_lag4,num_5star_two_weeks,num_5star_two_weeks_lag1,num_5star_two_weeks_lag2,num_5star_one_month,num_5star_two_months,num_5star_one_quarter,num_5star,avg_word_count,avg_word_count_10,perc_named_author,perc_verified,perc_verified_one_quarter,perc_verified_one_month,num_image,num_image_one_quarter,num_image_one_month,num_vine,num_vine_one_quarter,num_vine_one_month,avg_weekly_price,avg_monthly_price
0,amazon,2019-03-17 19:20:08,office-products/490930011,Binder Index Dividers,108.0,office-products/1069564,Binder Accessories,361.0,2019-03-31,2019-03-17,2019,11,0,2019_11,108.428571,4.682131,NaN,5.0,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,5.0,5.0,4.520325,123.0,1.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,3.0,3.0,3.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,7.0,1.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,3.0,3.0,3.0,93.0,15.569106,8.5,0.934959,0.878049,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,amazon,2019-03-24 22:27:37,office-products/490930011,Binder Index Dividers,119.0,office-products/1069564,Binder Accessories,391.0,2019-03-31,2019-03-24,2019,12,0,2019_12,124.428571,4.779123,0.096992,5.0,NaN,5.0,NaN,NaN,NaN,5.0,5.0,NaN,5.0,5.0,5.0,4.520325,123.0,0.0,1.0,NaN,NaN,NaN,1.0,2.0,NaN,2.0,3.0,3.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,7.0,0.0,1.0,NaN,NaN,NaN,1.0,2.0,NaN,2.0,3.0,3.0,93.0,15.569106,8.5,0.934959,0.878049,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,amazon,2019-03-31 20:58:18,office-products/490930011,Binder Index Dividers,105.0,office-products,Office Products,28193.0,2019-03-31,2019-03-31,2019,13,0,2019_13,130.428571,4.653960,-0.125163,5.0,NaN,NaN,5.0,NaN,NaN,NaN,5.0,5.0,5.0,5.0,5.0,4.520325,123.0,0.0,0.0,1.0,NaN,NaN,0.0,1.0,2.0,2.0,3.0,3.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,1.0,NaN,NaN,0.0,1.0,2.0,2.0,3.0,3.0,93.0,15.569106,8.5,0.934959,0.878049,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,amazon,2019-04-07 18:58:09,office-products/490930011,Binder Index Dividers,113.0,office-products,Office Products,33031.0,2019-04-30,2019-04-07,2019,14,0,2019_14,139.714286,4.727388,0.073427,5.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,5.0,5.0,5.0,5.0,4.520325,123.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,1.0,2.0,3.0,3.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,1.0,2.0,3.0,3.0,93.0,15.569106,8.5,0.934959,0.878049,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4,amazon,2019-04-14 22:53:41,office-products/490930011,Binder Index Dividers,86.0,office-products,Office Products,22984.0,2019-04-30,2019-04-14,2019,15,0,2019_15,134.000000,4.454347,-0.273041,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,5.0,5.0,4.520325,123.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,3.0,93.0,15.569106,8.5,0.934959,0.878049,1.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


### Export Weekly Features into dataframe and csv file

In [0]:
# add a column 'product' to the dataframe in the dictionary
for i in avery_sales_rank_dict_week:
    avery_sales_rank_dict_week[i]['product'] = i

In [0]:
avery_sales_rank_dict_week[products[0]].columns

Index(['source', 'date', 'category_id1', 'category_name1', 'category_rank1',
       'category_id2', 'category_name2', 'category_rank2', 'month', 'new_date',
       'year', 'week_of_year', 'weekday', 'year_and_week', 'avg_weekly_rank',
       'log_rank', 'delta_rank', 'stars_recent_10', 'stars_one_week',
       'stars_pri_1stweek', 'stars_pri_2ndweek', 'stars_pri_3rdweek',
       'stars_pri_4thweek', 'stars_two_weeks', 'stars_two_weeks_lag1',
       'stars_two_weeks_lag2', 'stars_one_month', 'stars_two_months',
       'stars_one_quarter', 'stars_avg', 'num_review', 'num_review_one_week',
       'num_review_one_week_lag1', 'num_review_one_week_lag2',
       'num_review_one_week_lag3', 'num_review_one_week_lag4',
       'num_review_two_weeks', 'num_review_two_weeks_lag1',
       'num_review_two_weeks_lag2', 'num_review_one_month',
       'num_review_two_months', 'num_review_one_quarter', 'num_1star_one_week',
       'num_1star_one_week_lag1', 'num_1star_one_week_lag2',
       'num_1star_o

In [0]:
# convert to dataframe
avery_weekly_features = pd.DataFrame()
for i in tqdm(avery_sales_rank_dict_week):
    if i in avery_review_dict:
        if type(avery_review_dict[i]) != pd.core.series.Series:
            df_bin = avery_sales_rank_dict_week[i][['product', 'source', 'date', 'category_id1', 'category_name1', 'category_rank1', 'month', 'new_date',
           'year', 'week_of_year', 'weekday', 'year_and_week', 'avg_weekly_rank', 'log_rank', 'delta_rank',
           'stars_recent_10', 'stars_one_week', 'stars_pri_1stweek',
           'stars_pri_2ndweek', 'stars_pri_3rdweek', 'stars_pri_4thweek',
           'stars_two_weeks', 'stars_two_weeks_lag1', 'stars_two_weeks_lag2',
           'stars_one_month', 'stars_two_months', 'stars_one_quarter', 'stars_avg',
           'num_review', 'num_review_one_week', 'num_review_one_week_lag1',
           'num_review_one_week_lag2', 'num_review_one_week_lag3',
           'num_review_one_week_lag4', 'num_review_two_weeks',
           'num_review_two_weeks_lag1', 'num_review_two_weeks_lag2',
           'num_review_one_month', 'num_review_two_months',
           'num_review_one_quarter', 'num_1star_one_week',
           'num_1star_one_week_lag1', 'num_1star_one_week_lag2',
           'num_1star_one_week_lag3', 'num_1star_one_week_lag4',
           'num_1star_two_weeks', 'num_1star_two_weeks_lag1',
           'num_1star_two_weeks_lag2', 'num_1star_one_month',
           'num_1star_two_months', 'num_1star_one_quarter', 'num_1star',
           'num_5star_one_week', 'num_5star_one_week_lag1',
           'num_5star_one_week_lag2', 'num_5star_one_week_lag3',
           'num_5star_one_week_lag4', 'num_5star_two_weeks',
           'num_5star_two_weeks_lag1', 'num_5star_two_weeks_lag2',
           'num_5star_one_month', 'num_5star_two_months', 'num_5star_one_quarter',
           'num_5star', 'avg_word_count', 'avg_word_count_10', 'perc_named_author',
           'perc_verified', 'perc_verified_one_quarter', 'perc_verified_one_month',
           'num_image', 'num_image_one_quarter', 'num_image_one_month', 'num_vine',
           'num_vine_one_quarter', 'num_vine_one_month', 'avg_weekly_price',
           'avg_monthly_price']]
        avery_weekly_features = pd.concat([avery_weekly_features, df_bin], ignore_index = True)

100%|██████████| 301/301 [00:01<00:00, 181.59it/s]


In [0]:
avery_weekly_features.shape

(11624, 78)

In [0]:
avery_weekly_features.drop_duplicates(inplace = True)

In [0]:
filename = 'Avery_'+re.sub(r'[-/]', '_', category_id_str)+'_weekly.csv'
avery_weekly_features.to_csv(filename)

## Aggregate Weekly-based Delta Delta Features

In [0]:
avery_data = avery_weekly_features
products = avery_data["product"].unique()
WoY = avery_data["year_and_week"].unique()

In [0]:
final_df = pd.DataFrame()
for i in tqdm(range(0, len(products))):
    for j in range(0, len(WoY)-1):
        # select all the rows in the current week from the avery_data
        week_df_bin = avery_data[(avery_data['product'] != products[i]) & \
                                 (avery_data['year_and_week'].isin(WoY[j:j+2])) &\
                                  (avery_data['year_and_week'] == WoY[j])].reset_index(drop = True)
        
        # extract the average weekly sales rank of the product in interest
        if (len(avery_data[(avery_data['product'] == products[i]) & \
                                (avery_data['year_and_week'] == WoY[j])]['avg_weekly_rank']) > 0) &\
        (avery_data[avery_data['product']==products[i]]['year_and_week'].isin([WoY[j+1]]).sum() > 0):
            input_rank = avery_data[(avery_data['product'] == products[i]) & \
                                    (avery_data['year_and_week'] == WoY[j])]['avg_weekly_rank'].reset_index(drop = True)[0]

            # select all the rows of the 10 products that have closest average weekly sales rank to the product in interest
            match_products = week_df_bin.iloc[(week_df_bin['avg_weekly_rank'] - input_rank).abs().argsort()[:10]]
            # calculate the log(average rank) of the 10 products in the current week
            log_rank_match_products = np.log(match_products['avg_weekly_rank'].mean())
            # calculate the log(average rank) of the 10 products in the next week
            products_bin = match_products['product'].unique()
            log_rank_match_products_next_week = np.log(avery_data[(avery_data['product'].isin(products_bin)) & \
                                                                   (avery_data['year_and_week'] == WoY[j+1])]['avg_weekly_rank'].mean())
            # calculate the delta(log(average rank)) of the 10 products by differencing the figures for two wks
            delta_log_rank_match_products = log_rank_match_products_next_week - log_rank_match_products

            # calculate the log(rank) of the product in interest in the current week
            log_rank = np.log(input_rank)
            # calculate the log(rank) of the product in interest in the next week
            log_rank_next_week = np.log(avery_data[(avery_data['product'] == products[i]) & \
                                (avery_data['year_and_week'] == WoY[j+1])]['avg_weekly_rank'].reset_index(drop = True)[0])
            # calculate the delta(log(rank)) of this product by differencing the figures for two wks
            delta_log_rank = log_rank_next_week - log_rank

            # finally, calculate the delta of delta(log(rank)) by differencing the growth of product in interest 
            # and the growth of 10 products
            delta_delta_log_rank = delta_log_rank - delta_log_rank_match_products
            delta_delta_log_rank = pd.DataFrame({"delta_delta_log_rank": [delta_delta_log_rank]})

            product_date_week = avery_data[(avery_data['product'] == products[i]) & \
                                       (avery_data['year_and_week'] == WoY[j])][['product', 'new_date', 'year_and_week']].reset_index(drop = True)
            
            delta_review_metrics = avery_data[(avery_data['product'] == products[i]) & \
                                              (avery_data['year_and_week'] == WoY[j])].iloc[0, 16:] - match_products.iloc[:, 16:].mean(axis = 0)
            delta_review_metrics = pd.DataFrame(delta_review_metrics).T
            df_bin = pd.concat([product_date_week, delta_delta_log_rank, delta_review_metrics], axis = 1)
            final_df = pd.concat([final_df, df_bin], ignore_index = True)

100%|██████████| 206/206 [08:29<00:00,  2.47s/it]


In [0]:
colnames = final_df.columns.tolist()
colnames

['product',
 'new_date',
 'year_and_week',
 'delta_delta_log_rank',
 'stars_one_week',
 'stars_pri_1stweek',
 'stars_pri_2ndweek',
 'stars_pri_3rdweek',
 'stars_pri_4thweek',
 'stars_two_weeks',
 'stars_two_weeks_lag1',
 'stars_two_weeks_lag2',
 'stars_one_month',
 'stars_two_months',
 'stars_one_quarter',
 'stars_avg',
 'num_review',
 'num_review_one_week',
 'num_review_one_week_lag1',
 'num_review_one_week_lag2',
 'num_review_one_week_lag3',
 'num_review_one_week_lag4',
 'num_review_two_weeks',
 'num_review_two_weeks_lag1',
 'num_review_two_weeks_lag2',
 'num_review_one_month',
 'num_review_two_months',
 'num_review_one_quarter',
 'num_1star_one_week',
 'num_1star_one_week_lag1',
 'num_1star_one_week_lag2',
 'num_1star_one_week_lag3',
 'num_1star_one_week_lag4',
 'num_1star_two_weeks',
 'num_1star_two_weeks_lag1',
 'num_1star_two_weeks_lag2',
 'num_1star_one_month',
 'num_1star_two_months',
 'num_1star_one_quarter',
 'num_1star',
 'num_5star_one_week',
 'num_5star_one_week_lag1',
 'n

In [0]:
final_df.shape

(10804, 66)

In [0]:
new_colnames = colnames[0:4]
for i in range(4, len(colnames)):
    new_colnames.append('delta_' + colnames[i])

In [0]:
final_df.columns = new_colnames

In [0]:
filename2 = 'Avery_weekly_delta_delta_log_' + re.sub(r'[-/]', '_', category_id_str) + '.csv'
final_df.to_csv(filename2)

## Generate Monthly Metrics

### Aggregate Sales Rank Features

In [0]:
for i in tqdm(avery_sales_rank_dict_month):
    if i in avery_review_dict:
        for j in avery_sales_rank_dict_month[i].index.unique():
            if type(avery_review_dict[i]) != pd.core.series.Series:
                avery_sales_rank_dict_month[i].loc[j, 'avg_monthly_rank'] = \
                avery_sales_rank_dict_daily[i][(avery_sales_rank_dict_daily[i]['month'] ==
                                              avery_sales_rank_dict_month[i].loc[j, 'month'])]['category_rank1'].mean()
                avery_sales_rank_dict_month[i]['log_rank'] = \
                avery_sales_rank_dict_month[i]['category_rank1'].map(lambda x: math.log(x))
                avery_sales_rank_dict_month[i]['delta_rank'] = avery_sales_rank_dict_month[i]['log_rank'].diff()

100%|████████████████████████████████████████████████████████████████████████████████| 425/425 [00:04<00:00, 95.85it/s]


### Aggregate Star Rating Features

In [0]:
# stars_one_month: the average star rating of the most recent one month
for i in tqdm(avery_sales_rank_dict_month):
    if i in avery_review_dict:
        for j in avery_sales_rank_dict_month[i].index.unique():
            if type(avery_review_dict[i]) != pd.core.series.Series:
                
                # average star ratings of the most recent 10 reviews by the target date
                avery_sales_rank_dict_month[i].loc[j, 'stars_recent_10'] = \
                avery_review_dict[i][avery_review_dict[i]['new_date'] <= \
                                     avery_sales_rank_dict_month[i].loc[j, 'new_date']].tail(10)['stars'].mean()
                
                # average star ratings in recent one week
                avery_sales_rank_dict_month[i].loc[j, "stars_one_week"] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_month[i].loc[j, 'new_date']) &\
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 7)))]['stars'].mean()
                
                # stars_pri_1stweek: the average star rating in the first week prior to this week
                # stars_pri_1stweek is also the stars_one_week lagged by one - not in month's case
                avery_sales_rank_dict_month[i]['stars_pri_1stweek'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < (avery_sales_rank_dict_month[i].loc[j, 'new_date'] - \
                                                                           timedelta(days = 7))) & \
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 14)))]['stars'].mean()
                avery_sales_rank_dict_month[i]['stars_pri_2ndweek'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < (avery_sales_rank_dict_month[i].loc[j, 'new_date'] - \
                                                                           timedelta(days = 14))) & \
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 21)))]['stars'].mean()
                avery_sales_rank_dict_month[i]['stars_pri_3rdweek'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < (avery_sales_rank_dict_month[i].loc[j, 'new_date'] - \
                                                                           timedelta(days = 21))) & \
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 28)))]['stars'].mean()
                avery_sales_rank_dict_month[i]['stars_pri_4thweek'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < (avery_sales_rank_dict_month[i].loc[j, 'new_date'] - \
                                                                           timedelta(days = 28))) & \
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 35)))]['stars'].mean()              
                

100%|████████████████████████████████████████████████████████████████████████████████| 425/425 [00:31<00:00, 13.47it/s]


In [0]:
# stars_two_weeks: the average star rating of the most recent two weeks
for i in tqdm(avery_sales_rank_dict_month):
    if i in avery_review_dict:
        for j in avery_sales_rank_dict_month[i].index.unique():
            if type(avery_review_dict[i]) != pd.core.series.Series:
                avery_sales_rank_dict_month[i].loc[j, "stars_two_weeks"] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_month[i].loc[j, 'new_date']) &\
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 14)))]['stars'].mean()
                avery_sales_rank_dict_month[i]['stars_two_weeks_lag1'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < (avery_sales_rank_dict_month[i].loc[j, 'new_date'] - \
                                                                          timedelta(days = 14))) &\
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 28)))]['stars'].mean()
                avery_sales_rank_dict_month[i]['stars_two_weeks_lag2'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < (avery_sales_rank_dict_month[i].loc[j, 'new_date'] - \
                                                                          timedelta(days = 28))) &\
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 42)))]['stars'].mean()

100%|████████████████████████████████████████████████████████████████████████████████| 425/425 [00:19<00:00, 22.31it/s]


In [0]:
for i in tqdm(avery_sales_rank_dict_month):
    if i in avery_review_dict:
        for j in avery_sales_rank_dict_month[i].index.unique():
            if type(avery_review_dict[i]) != pd.core.series.Series:
                avery_sales_rank_dict_month[i].loc[j, "stars_one_month"] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_month[i].loc[j, 'new_date']) &\
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 30)))]['stars'].mean()
                avery_sales_rank_dict_month[i].loc[j, "stars_two_months"] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_month[i].loc[j, 'new_date']) &\
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 60)))]['stars'].mean()
                avery_sales_rank_dict_month[i].loc[j, "stars_one_quarter"] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_month[i].loc[j, 'new_date']) &\
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 90)))]['stars'].mean()
                avery_sales_rank_dict_month[i].loc[j, "stars_avg"] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_month[i].loc[j, 'new_date'])]['stars'].mean()
             

100%|████████████████████████████████████████████████████████████████████████████████| 425/425 [00:24<00:00, 17.55it/s]


In [0]:
avery_sales_rank_dict_month[products[0]].head()
# results showing the data for the previous month?

,source,date,category_id1,category_name1,category_rank1,category_id2,category_name2,category_rank2,month,new_date,...,stars_one_week,stars_pri_1stweek,stars_pri_2ndweek,stars_pri_3rdweek,stars_pri_4thweek,stars_two_weeks,stars_one_month,stars_two_months,stars_one_quarter,stars_avg
0,amazon,2019-04-01 21:57:55,office-products/1069410,Laser & Inkjet Printer Labels,2.0,office-products,Office Products,573.0,2019-04-01,2019-04-01,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.000000,4.555556,4.612360
1,amazon,2019-05-01 20:14:41,office-products/1069410,Laser & Inkjet Printer Labels,6.0,office-products,Office Products,742.0,2019-05-01,2019-05-01,...,NaN,NaN,NaN,NaN,5.0,2.00,3.833333,3.833333,4.416667,4.603704
2,amazon,2019-06-01 19:26:18,office-products/1069410,Laser & Inkjet Printer Labels,7.0,office-products,Office Products,914.0,2019-06-01,2019-06-01,...,NaN,NaN,NaN,NaN,5.0,3.50,4.000000,3.888889,3.888889,4.600368
3,amazon,2019-07-01 19:52:08,office-products/1069410,Laser & Inkjet Printer Labels,7.0,office-products,Office Products,819.0,2019-07-01,2019-07-01,...,5.00,NaN,NaN,NaN,5.0,5.00,5.000000,4.500000,4.166667,4.602564
4,amazon,2019-08-01 19:08:09,office-products/1069410,Laser & Inkjet Printer Labels,14.0,office-products,Office Products,1611.0,2019-08-01,2019-08-01,...,4.25,NaN,NaN,NaN,5.0,4.25,4.727273,4.785714,4.647059,4.605027


### Aggregate Quantity of Reviews Features

In [0]:
for i in tqdm(avery_sales_rank_dict_month):
    if i in avery_review_dict:
        for j in avery_sales_rank_dict_month[i].index.unique():
            if type(avery_review_dict[i]) != pd.core.series.Series:
                avery_sales_rank_dict_month[i].loc[j, "num_review"] = \
                avery_review_dict[i][avery_review_dict[i]['new_date'] <= \
                                     avery_sales_rank_dict_month[i].loc[j, 'new_date']]['new_date'].count()
               
                avery_sales_rank_dict_month[i].loc[j, 'num_review_one_week'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                       timedelta(days = 7)))]['new_date'].count()
                
                avery_sales_rank_dict_month[i]['num_review_one_week_lag1'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < (avery_sales_rank_dict_month[i].loc[j, 'new_date'] - \
                                                                           timedelta(days = 7))) & \
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 14)))]['stars'].count()
                avery_sales_rank_dict_month[i]['num_review_one_week_lag2'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < (avery_sales_rank_dict_month[i].loc[j, 'new_date'] - \
                                                                           timedelta(days = 14))) & \
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 21)))]['stars'].count()
                avery_sales_rank_dict_month[i]['num_review_one_week_lag3'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < (avery_sales_rank_dict_month[i].loc[j, 'new_date'] - \
                                                                           timedelta(days = 21))) & \
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 28)))]['stars'].count()
                avery_sales_rank_dict_month[i]['num_review_one_week_lag4'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < (avery_sales_rank_dict_month[i].loc[j, 'new_date'] - \
                                                                           timedelta(days = 28))) & \
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 35)))]['stars'].count()

                
                avery_sales_rank_dict_month[i].loc[j, "num_review_two_weeks"] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_month[i].loc[j, 'new_date']) &\
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 14)))]['stars'].count()
                avery_sales_rank_dict_month[i]['num_review_two_weeks_lag1'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < (avery_sales_rank_dict_month[i].loc[j, 'new_date'] - \
                                                                          timedelta(days = 14))) &\
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 28)))]['stars'].count()
                avery_sales_rank_dict_month[i]['num_review_two_weeks_lag2'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < (avery_sales_rank_dict_month[i].loc[j, 'new_date'] - \
                                                                          timedelta(days = 28))) &\
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 42)))]['stars'].count()

                avery_sales_rank_dict_month[i].loc[j, "num_review_one_month"] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_month[i].loc[j, 'new_date']) &\
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 30)))]['stars'].count()
                avery_sales_rank_dict_month[i].loc[j, "num_review_two_months"] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_month[i].loc[j, 'new_date']) &\
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 60)))]['stars'].count()
                avery_sales_rank_dict_month[i].loc[j, "num_review_one_quarter"] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_month[i].loc[j, 'new_date']) &\
                                     (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                                                           timedelta(days = 90)))]['stars'].count()

100%|████████████████████████████████████████████████████████████████████████████████| 425/425 [01:10<00:00,  6.03it/s]


In [0]:
avery_sales_rank_dict_month[products[0]].head()

,source,date,category_id1,category_name1,category_rank1,category_id2,category_name2,category_rank2,month,new_date,...,num_review_one_week_lag1,num_review_one_week_lag2,num_review_one_week_lag3,num_review_one_week_lag4,num_review_two_weeks,num_review_two_weeks_lag1,num_review_two_weeks_lag2,num_review_one_month,num_review_two_months,num_review_one_quarter
0,amazon,2019-03-31 21:16:31,office-products/1069410,Laser & Inkjet Printer Labels,2.0,office-products,Office Products,636.0,2019-03-31,2019-03-31,...,0,0,3,2,0.0,3,3,1.0,6.0,9.0
1,amazon,2019-04-30 19:03:28,office-products/1069410,Laser & Inkjet Printer Labels,5.0,office-products,Office Products,785.0,2019-04-30,2019-04-30,...,0,0,3,2,2.0,3,3,6.0,7.0,12.0
2,amazon,2019-05-31 22:42:03,office-products/1069410,Laser & Inkjet Printer Labels,4.0,office-products,Office Products,827.0,2019-05-31,2019-05-31,...,0,0,3,2,2.0,3,3,3.0,9.0,9.0
3,amazon,2019-06-30 19:28:32,office-products/1069410,Laser & Inkjet Printer Labels,7.0,office-products,Office Products,733.0,2019-06-30,2019-06-30,...,0,0,3,2,2.0,3,3,3.0,6.0,12.0
4,amazon,2019-07-31 19:43:45,office-products/1069410,Laser & Inkjet Printer Labels,19.0,office-products,Office Products,1865.0,2019-07-31,2019-07-31,...,0,0,3,2,4.0,3,3,11.0,14.0,17.0


### Aggregate Quantity of Reviews with Extreme Ratings

In [0]:
for i in tqdm(avery_sales_rank_dict_month):
    if i in avery_review_dict:
        for j in avery_sales_rank_dict_month[i].index.unique():
            if type(avery_review_dict[i]) != pd.core.series.Series:
                avery_sales_rank_dict_month[i].loc[j, 'num_1star_one_week'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 7))) & \
                                     (avery_review_dict[i]['stars'] == 1)]['new_date'].count()
                
                avery_sales_rank_dict_month[i]['num_1star_one_week_lag1'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 7))) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 14))) & \
                                     (avery_review_dict[i]['stars'] == 1)]['new_date'].count()
                avery_sales_rank_dict_month[i]['num_1star_one_week_lag2'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 14))) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 21))) & \
                                     (avery_review_dict[i]['stars'] == 1)]['new_date'].count()
                avery_sales_rank_dict_month[i]['num_1star_one_week_lag3'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 21))) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 28))) & \
                                     (avery_review_dict[i]['stars'] == 1)]['new_date'].count()
                avery_sales_rank_dict_month[i]['num_1star_one_week_lag4'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 28))) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 35))) & \
                                     (avery_review_dict[i]['stars'] == 1)]['new_date'].count()
                
                avery_sales_rank_dict_month[i].loc[j, 'num_1star_two_weeks'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 14))) & \
                                     (avery_review_dict[i]['stars'] == 1)]['new_date'].count()
                
                avery_sales_rank_dict_month[i]['num_1star_two_weeks_lag1'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      (avery_sales_rank_dict_month[i].loc[j, 'new_date']- timedelta(days = 14))) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 28))) & \
                                     (avery_review_dict[i]['stars'] == 1)]['new_date'].count()
                avery_sales_rank_dict_month[i]['num_1star_two_weeks_lag2'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      (avery_sales_rank_dict_month[i].loc[j, 'new_date']- timedelta(days = 28))) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 42))) & \
                                     (avery_review_dict[i]['stars'] == 1)]['new_date'].count()
                
                avery_sales_rank_dict_month[i].loc[j, 'num_1star_one_month'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 30))) & \
                                     (avery_review_dict[i]['stars'] == 1)]['new_date'].count()
          
                avery_sales_rank_dict_month[i].loc[j, 'num_1star_two_months'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 60))) & \
                                     (avery_review_dict[i]['stars'] == 1)]['new_date'].count()
                
                avery_sales_rank_dict_month[i].loc[j, 'num_1star_one_quarter'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 90))) & \
                                     (avery_review_dict[i]['stars'] == 1)]['new_date'].count()
                
                avery_sales_rank_dict_month[i].loc[j, 'num_1star'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['stars'] == 1)]['new_date'].count()
                
                # quantity of 5-star reviews
                avery_sales_rank_dict_month[i].loc[j, 'num_5star_one_week'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 7))) & \
                                     (avery_review_dict[i]['stars'] == 5)]['new_date'].count()
                
                avery_sales_rank_dict_month[i]['num_5star_one_week_lag1'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 7))) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 14))) & \
                                     (avery_review_dict[i]['stars'] == 5)]['new_date'].count()
                avery_sales_rank_dict_month[i]['num_5star_one_week_lag2'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 14))) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 21))) & \
                                     (avery_review_dict[i]['stars'] == 5)]['new_date'].count()
                avery_sales_rank_dict_month[i]['num_5star_one_week_lag3'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 21))) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 28))) & \
                                     (avery_review_dict[i]['stars'] == 5)]['new_date'].count()
                avery_sales_rank_dict_month[i]['num_5star_one_week_lag4'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 28))) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 35))) & \
                                     (avery_review_dict[i]['stars'] == 5)]['new_date'].count()
                
                avery_sales_rank_dict_month[i].loc[j, 'num_5star_two_weeks'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 14))) & \
                                     (avery_review_dict[i]['stars'] == 5)]['new_date'].count()
                
                avery_sales_rank_dict_month[i]['num_5star_two_weeks_lag1'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      (avery_sales_rank_dict_month[i].loc[j, 'new_date']- timedelta(days = 14))) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 28))) & \
                                     (avery_review_dict[i]['stars'] == 5)]['new_date'].count()
                avery_sales_rank_dict_month[i]['num_5star_two_weeks_lag2'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      (avery_sales_rank_dict_month[i].loc[j, 'new_date']- timedelta(days = 28))) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 42))) & \
                                     (avery_review_dict[i]['stars'] == 5)]['new_date'].count()
                
                avery_sales_rank_dict_month[i].loc[j, 'num_5star_one_month'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 30))) & \
                                     (avery_review_dict[i]['stars'] == 5)]['new_date'].count()
          
                avery_sales_rank_dict_month[i].loc[j, 'num_5star_two_months'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 60))) & \
                                     (avery_review_dict[i]['stars'] == 5)]['new_date'].count()
                
                avery_sales_rank_dict_month[i].loc[j, 'num_5star_one_quarter'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['new_date'] >= \
                                      (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 90))) & \
                                     (avery_review_dict[i]['stars'] == 5)]['new_date'].count()
                
                avery_sales_rank_dict_month[i].loc[j, 'num_5star'] = \
                avery_review_dict[i][(avery_review_dict[i]['new_date'] < \
                                      avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                                     (avery_review_dict[i]['stars'] == 5)]['new_date'].count()
                

100%|████████████████████████████████████████████████████████████████████████████████| 425/425 [03:18<00:00,  2.14it/s]


In [0]:
avery_sales_rank_dict_month[products[0]].head()

,source,date,category_id1,category_name1,category_rank1,category_id2,category_name2,category_rank2,month,new_date,...,num_5star_one_week_lag2,num_5star_one_week_lag3,num_5star_one_week_lag4,num_5star_two_weeks,num_5star_two_weeks_lag1,num_5star_two_weeks_lag2,num_5star_one_month,num_5star_two_months,num_5star_one_quarter,num_5star
0,amazon,2019-04-01 21:57:55,office-products/1069410,Laser & Inkjet Printer Labels,2.0,office-products,Office Products,573.0,2019-04-01,2019-04-01,...,0,0,1,0.0,0,1,0.0,6.0,8.0,419.0
1,amazon,2019-05-01 20:14:41,office-products/1069410,Laser & Inkjet Printer Labels,6.0,office-products,Office Products,742.0,2019-05-01,2019-05-01,...,0,0,1,0.0,0,1,4.0,4.0,10.0,423.0
2,amazon,2019-06-01 19:26:18,office-products/1069410,Laser & Inkjet Printer Labels,7.0,office-products,Office Products,914.0,2019-06-01,2019-06-01,...,0,0,1,1.0,0,1,2.0,6.0,6.0,425.0
3,amazon,2019-07-01 19:52:08,office-products/1069410,Laser & Inkjet Printer Labels,7.0,office-products,Office Products,819.0,2019-07-01,2019-07-01,...,0,0,1,2.0,0,1,3.0,5.0,9.0,428.0
4,amazon,2019-08-01 19:08:09,office-products/1069410,Laser & Inkjet Printer Labels,14.0,office-products,Office Products,1611.0,2019-08-01,2019-08-01,...,0,0,1,3.0,0,1,10.0,13.0,15.0,438.0


### Aggregate Quality of Reviews Features

In [0]:
for i in tqdm(avery_sales_rank_dict_month):
    if i in avery_review_dict:
        for j in avery_sales_rank_dict_month[i].index.unique():
            if type(avery_review_dict[i]) != pd.core.series.Series:
                # aggregate word count related features
                wc_bin = avery_review_dict[i][avery_review_dict[i]['new_date'] <= \
                        avery_sales_rank_dict_month[i].loc[j, 'new_date']]['text'].tolist()
                wc_bin_str = ''.join(map(str, wc_bin))
                if len(wc_bin) > 0:
                    avery_sales_rank_dict_month[i].loc[j, 'avg_word_count'] = \
                    len(wc_bin_str.split())/len(wc_bin)
                else: 
                    avery_sales_rank_dict_month[i].loc[j, 'avg_word_count'] = 0
                    
                #  'avg_word_count_10' is the average word count of the recent 10 reviews
                wc2_bin = avery_review_dict[i][avery_review_dict[i]['new_date'] <= \
                    avery_sales_rank_dict_month[i].loc[j, 'new_date']].tail(10)['text'].tolist()
                wc2_bin_str = ''.join(map(str, wc2_bin))
                if len(wc2_bin) > 0:
                    avery_sales_rank_dict_month[i].loc[j, 'avg_word_count_10'] = \
                    len(wc2_bin_str.split())/len(wc2_bin)
                else:
                    avery_sales_rank_dict_month[i].loc[j, 'avg_word_count_10'] = 0
                
                # perc_named_author: the percentage of reviews written by non-anonymous authors to the date
                avery_sales_rank_dict_month[i].loc[j, 'perc_named_author'] = \
                avery_review_dict[i][
                    (avery_review_dict[i]['new_date'] <= avery_sales_rank_dict_month[i].loc[j,'new_date']) & \
                    (avery_review_dict[i]['author'].str.contains('[Cc]ustomer') == False)
                ]['new_date'].count() \
                    / avery_review_dict[i][avery_review_dict[i]['new_date'] \
                                <= avery_sales_rank_dict_month[i].loc[j,'new_date']]['new_date'].count()
                
                # 'perc_verified': the percentage of verified reviews to the date
                avery_sales_rank_dict_month[i].loc[j, 'perc_verified'] = \
                    (avery_review_dict[i][
                    (avery_review_dict[i]['new_date'] <= avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['verified'] == True)]['new_date'].count()) / \
                    (avery_review_dict[i][avery_review_dict[i]['new_date'] < \
                                        avery_sales_rank_dict_month[i].loc[j, 'new_date']]['new_date'].count()) 
                
                avery_sales_rank_dict_month[i].loc[j, 'perc_verified_one_quarter'] = \
                    (avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 90))) & \
                                          (avery_review_dict[i]['verified'] == True)]['new_date'].count()) / \
                    (avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                                          timedelta(days = 90)))]['new_date'].count()) 
            
                avery_sales_rank_dict_month[i].loc[j, 'perc_verified_one_month'] = \
                    (avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 30))) & \
                                          (avery_review_dict[i]['verified'] == True)]['new_date'].count()) / \
                    (avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - \
                                                          timedelta(days = 30)))]['new_date'].count()) 
                
                # 'num_image': number of reviews with image
                avery_sales_rank_dict_month[i].loc[j, 'num_image'] = \
                    (avery_review_dict[i][
                    (avery_review_dict[i]['new_date'] <= avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['image'] == 'Yes')]['new_date'].count())
                
                avery_sales_rank_dict_month[i].loc[j, 'num_image_one_quarter'] = \
                    (avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 90))) & \
                    (avery_review_dict[i]['image'] == 'Yes')]['new_date'].count())
                
                avery_sales_rank_dict_month[i].loc[j, 'num_image_one_month'] = \
                    (avery_review_dict[i][(avery_review_dict[i]['new_date'] < avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 30))) & \
                    (avery_review_dict[i]['image'] == 'Yes')]['new_date'].count())
                
                # 'num_vine': number of reviews by Vine customers
                avery_sales_rank_dict_month[i].loc[j, 'num_vine'] = \
                    (avery_review_dict[i][
                    (avery_review_dict[i]['new_date'] <= avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['vine'] == True)]['new_date'].count())
                
                avery_sales_rank_dict_month[i].loc[j, 'num_vine_one_quarter'] = \
                    (avery_review_dict[i][(
                    avery_review_dict[i]['new_date'] < avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 90))) & \
                    (avery_review_dict[i]['vine'] == True)]['new_date'].count())
                
                avery_sales_rank_dict_month[i].loc[j, 'num_vine_one_month'] = \
                    (avery_review_dict[i][(
                    avery_review_dict[i]['new_date'] < avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                    (avery_review_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 30))) & \
                    (avery_review_dict[i]['vine'] == True)]['new_date'].count())
                

 56%|████████████████████████████████████████████▊                                   | 238/425 [01:18<00:59,  3.15it/s]C:\Users\nora_\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in long_scalars
C:\Users\nora_\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in long_scalars
100%|████████████████████████████████████████████████████████████████████████████████| 425/425 [01:55<00:00,  3.68it/s]


### Aggregate Price Features

In [0]:
for i in tqdm(avery_sales_rank_dict_month):
    if i in avery_review_dict:
        for j in avery_sales_rank_dict_month[i].index.unique():
            if type(avery_review_dict[i]) != pd.core.series.Series:
                avery_sales_rank_dict_month[i].loc[j, 'avg_monthly_price'] = avery_price_dict[i][(\
                    avery_price_dict[i]['new_date'] < avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                    (avery_price_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 30)))\
                                                                                                ]['price'].mean()
                avery_sales_rank_dict_month[i].loc[j, 'avg_quarterly_price'] = avery_price_dict[i][(\
                    avery_price_dict[i]['new_date'] < avery_sales_rank_dict_month[i].loc[j, 'new_date']) & \
                    (avery_price_dict[i]['new_date'] >= (avery_sales_rank_dict_month[i].loc[j,'new_date'] - timedelta(days = 90)))\
                                                                                                ]['price'].mean()

100%|████████████████████████████████████████████████████████████████████████████████| 425/425 [00:13<00:00, 32.39it/s]


In [0]:
avery_sales_rank_dict_month[products[0]].head(15)

,source,date,category_id1,category_name1,category_rank1,category_id2,category_name2,category_rank2,month,new_date,...,perc_verified_one_month,num_image,num_image_one_quarter,num_image_one_month,num_vine,num_vine_one_quarter,num_vine_one_month,avg_monthly_price,avg_quarterly_price,product
0,amazon,2019-03-31 21:16:31,office-products/1069410,Laser & Inkjet Printer Labels,2.0,office-products,Office Products,636.0,2019-03-31,2019-03-31,...,1.000000,2.0,0.0,0.0,29.0,0.0,0.0,8.370333,8.370333,B00004Z5QO
1,amazon,2019-04-30 19:03:28,office-products/1069410,Laser & Inkjet Printer Labels,5.0,office-products,Office Products,785.0,2019-04-30,2019-04-30,...,0.833333,2.0,0.0,0.0,29.0,0.0,0.0,8.282000,8.326167,B00004Z5QO
2,amazon,2019-05-31 22:42:03,office-products/1069410,Laser & Inkjet Printer Labels,4.0,office-products,Office Products,827.0,2019-05-31,2019-05-31,...,1.000000,2.0,0.0,0.0,29.0,0.0,0.0,8.260000,8.301667,B00004Z5QO
3,amazon,2019-06-30 19:28:32,office-products/1069410,Laser & Inkjet Printer Labels,7.0,office-products,Office Products,733.0,2019-06-30,2019-06-30,...,1.000000,2.0,0.0,0.0,29.0,0.0,0.0,8.300667,8.278444,B00004Z5QO
4,amazon,2019-07-31 19:43:45,office-products/1069410,Laser & Inkjet Printer Labels,19.0,office-products,Office Products,1865.0,2019-07-31,2019-07-31,...,1.000000,3.0,1.0,1.0,29.0,0.0,0.0,9.480000,8.671236,B00004Z5QO
5,amazon,2019-08-31 20:34:14,office-products/1069410,Laser & Inkjet Printer Labels,10.0,office-products,Office Products,1702.0,2019-08-31,2019-08-31,...,1.000000,3.0,1.0,0.0,29.0,0.0,0.0,9.480000,9.091818,B00004Z5QO
6,amazon,2019-09-30 20:57:32,office-products/1069410,Laser & Inkjet Printer Labels,15.0,office-products,Office Products,1448.0,2019-09-30,2019-09-30,...,1.000000,3.0,1.0,0.0,29.0,0.0,0.0,9.480000,9.480000,B00004Z5QO
7,amazon,2019-10-31 19:55:50,office-products/1069410,Laser & Inkjet Printer Labels,11.0,office-products,Office Products,778.0,2019-10-31,2019-10-31,...,1.000000,3.0,0.0,0.0,29.0,0.0,0.0,8.511538,9.180238,B00004Z5QO
8,amazon,2019-11-30 21:36:32,office-products/1069410,Laser & Inkjet Printer Labels,11.0,office-products,Office Products,1776.0,2019-11-30,2019-11-30,...,1.000000,3.0,0.0,0.0,29.0,0.0,0.0,7.575862,8.522857,B00004Z5QO
9,amazon,2019-12-31 21:21:28,office-products/1069410,Laser & Inkjet Printer Labels,7.0,office-products,Office Products,986.0,2019-12-31,2019-12-31,...,1.000000,3.0,0.0,0.0,29.0,0.0,0.0,6.895517,7.608313,B00004Z5QO


### Export Monthly Features into dataframe and csv file

In [0]:
# add a column 'product' to the dataframe in the dictionary
for i in avery_sales_rank_dict_month:
    avery_sales_rank_dict_month[i]['product'] = i

In [0]:
avery_sales_rank_dict_month[products[0]].columns

Index(['source', 'date', 'category_id1', 'category_name1', 'category_rank1',
       'category_id2', 'category_name2', 'category_rank2', 'month', 'new_date',
       'year', 'week_of_year', 'weekday', 'year_and_week', 'log_rank',
       'delta_rank', 'stars_recent_10', 'stars_one_week', 'stars_pri_1stweek',
       'stars_pri_2ndweek', 'stars_pri_3rdweek', 'stars_pri_4thweek',
       'stars_two_weeks', 'stars_two_weeks_lag1', 'stars_two_weeks_lag2',
       'stars_one_month', 'stars_two_months', 'stars_one_quarter', 'stars_avg',
       'num_review', 'num_review_one_week', 'num_review_one_week_lag1',
       'num_review_one_week_lag2', 'num_review_one_week_lag3',
       'num_review_one_week_lag4', 'num_review_two_weeks',
       'num_review_two_weeks_lag1', 'num_review_two_weeks_lag2',
       'num_review_one_month', 'num_review_two_months',
       'num_review_one_quarter', 'num_1star_one_week',
       'num_1star_one_week_lag1', 'num_1star_one_week_lag2',
       'num_1star_one_week_lag3', 'num

In [0]:
# convert to dataframe
avery_monthly_features = pd.DataFrame()
for i in tqdm(avery_sales_rank_dict_month):
    if i in avery_review_dict:
        if type(avery_review_dict[i]) != pd.core.series.Series:
            df_bin = avery_sales_rank_dict_month[i][['product', 'source', 'date', 'category_id1', 'category_name1', 'category_rank1', 'month', 'new_date',
           'year', 'week_of_year', 'weekday', 'year_and_week', 'log_rank', 'delta_rank','avg_monthly_rank',
           'stars_recent_10', 'stars_one_week', 'stars_pri_1stweek',
           'stars_pri_2ndweek', 'stars_pri_3rdweek', 'stars_pri_4thweek',
           'stars_two_weeks', 'stars_two_weeks_lag1', 'stars_two_weeks_lag2',
           'stars_one_month', 'stars_two_months', 'stars_one_quarter', 'stars_avg',
           'num_review', 'num_review_one_week', 'num_review_one_week_lag1',
           'num_review_one_week_lag2', 'num_review_one_week_lag3',
           'num_review_one_week_lag4', 'num_review_two_weeks',
           'num_review_two_weeks_lag1', 'num_review_two_weeks_lag2',
           'num_review_one_month', 'num_review_two_months',
           'num_review_one_quarter', 'num_1star_one_week',
           'num_1star_one_week_lag1', 'num_1star_one_week_lag2',
           'num_1star_one_week_lag3', 'num_1star_one_week_lag4',
           'num_1star_two_weeks', 'num_1star_two_weeks_lag1',
           'num_1star_two_weeks_lag2', 'num_1star_one_month',
           'num_1star_two_months', 'num_1star_one_quarter', 'num_1star',
           'num_5star_one_week', 'num_5star_one_week_lag1',
           'num_5star_one_week_lag2', 'num_5star_one_week_lag3',
           'num_5star_one_week_lag4', 'num_5star_two_weeks',
           'num_5star_two_weeks_lag1', 'num_5star_two_weeks_lag2',
           'num_5star_one_month', 'num_5star_two_months', 'num_5star_one_quarter',
           'num_5star', 'avg_word_count', 'avg_word_count_10', 'perc_named_author',
           'perc_verified', 'perc_verified_one_quarter', 'perc_verified_one_month',
           'num_image', 'num_image_one_quarter', 'num_image_one_month', 'num_vine',
           'num_vine_one_quarter', 'num_vine_one_month',
           'avg_monthly_price','avg_quarterly_price']]
        avery_monthly_features = pd.concat([avery_monthly_features, df_bin], ignore_index = True)

100%|███████████████████████████████████████████████████████████████████████████████| 425/425 [00:02<00:00, 163.02it/s]


In [0]:
avery_monthly_features.shape

(4474, 78)

In [0]:
avery_monthly_features.drop_duplicates(inplace = True)

In [0]:
avery_monthly_features.to_csv('Avery_monthly_office_products_1069410.csv',index=False)

## Aggregate Monthly-based Delta Delta Features

In [0]:
avery_data = avery_monthly_features
products = avery_data["product"].unique()
months = avery_data["month"].unique()

In [0]:
final_df = pd.DataFrame()
for i in tqdm(range(0, len(products))): 
    for j in range(0, len(months)-1):
        # select rows of other products that have both the current month and the consecutive month's records from the avery_data
        month_df_bin = avery_data[(avery_data['product'] != products[i]) & \
                            (avery_data['month'].isin(months[j:j+2])) & \
                          (avery_data['month'] == months[j])].reset_index(drop = True)
        
        # extract the average monthly sales rank of the product in interest
        if (len(avery_data[(avery_data['product'] == products[i]) & \
                                (avery_data['month'] == months[j])]['avg_monthly_rank']) > 0) & \
        (avery_data[avery_data['product']==products[i]]['month'].isin([months[j+1]]).sum() > 0): 
        # ensure product itself also has records in the next month
            
            input_rank = avery_data[(avery_data['product'] == products[i]) & \
                                    (avery_data['month'] == months[j])]['avg_monthly_rank'].reset_index(drop = True)[0]

            # select all the rows of the 10 products that have closest average monthly sales rank to the product in interest
            match_products = month_df_bin.iloc[(month_df_bin['avg_monthly_rank'] - input_rank).abs().argsort()[:10]]
            # calculate the log(average rank) of the 10 products in the current month
            log_rank_match_products = np.log(match_products['avg_monthly_rank'].mean())
            # calculate the log(average rank) of the 10 products in the next month
            products_bin = match_products['product'].unique()
            log_rank_match_products_next_month = np.log(avery_data[(avery_data['product'].isin(products_bin)) & \
                                                                   (avery_data['month'] == months[j+1])]['avg_monthly_rank'].mean())
            # calculate the delta(log(average rank)) of the 10 products by differencing the figures for two months
            delta_log_rank_match_products = log_rank_match_products_next_month - log_rank_match_products

            # calculate the log(rank) of the product in interest in the current month
            log_rank = np.log(input_rank)
            # calculate the log(rank) of the product in interest in the next month
            log_rank_next_month = np.log(avery_data[(avery_data['product'] == products[i]) & \
                                (avery_data['month'] == months[j+1])]['avg_monthly_rank'].reset_index(drop = True)[0])
            # calculate the delta(log(rank)) of this product by differencing the figures for two months
            delta_log_rank = log_rank_next_month - log_rank

            # finally, calculate the delta of delta(log(rank)) by differencing the growth of product in interest 
            # and the growth of 10 products
            delta_delta_log_rank = delta_log_rank - delta_log_rank_match_products
            delta_delta_log_rank = pd.DataFrame({"delta_delta_log_rank": [delta_delta_log_rank]})

            product_month = avery_data[(avery_data['product'] == products[i]) & \
                                       (avery_data['month'] == months[j])][['product', 'month']].reset_index(drop = True)
            
            delta_review_metrics = avery_data[(avery_data['product'] == products[i]) & \
                                              (avery_data['month'] == months[j])].iloc[0, 12:78] - match_products.iloc[:, 12:78].mean(axis = 0)
            delta_review_metrics = pd.DataFrame(delta_review_metrics).T
            df_bin = pd.concat([product_month,  delta_delta_log_rank, delta_review_metrics], axis = 1)
            final_df = pd.concat([final_df, df_bin], ignore_index = True)

In [0]:
final_df.drop(columns='avg_monthly_rank',inplace=True)

In [0]:
final_df.columns.unique()

In [0]:
colnames = final_df.columns.tolist()
new_colnames = colnames[0:3]
for i in range(3, len(colnames)):
    new_colnames.append('delta_' + colnames[i])

In [0]:
final_df.columns = new_colnames

In [0]:
final_df.to_csv("Avery_monthly_delta_delta_log_office_products_1069410.csv")